# Strands Agents + Amazon Bedrock AgentCore Browser Tool CAPTCHA Handling

This tutorial demonstrates how Strands agents handle CAPTCHAs through Amazon Bedrock's AgentCore Browser Tool - a secure, serverless browser runtime designed specifically for AI agents.

## What is Amazon Bedrock AgentCore Browser Tool?

AgentCore Browser Tool is AWS's enterprise-grade, serverless browser runtime that provides:
- **Fully Managed**: AWS handles provisioning, patching, and scaling automatically
- **Enterprise Security**: Isolated VMs prevent data leakage between sessions
- **Auto-Scaling**: From single session to thousands based on demand
- **Built-in Observability**: Live viewing, CloudTrail logging, session replay
- **Agentic Framework Integration**: Native support for frameworks like Strands
- **Consumption-Based Pricing**: Pay only for what you use

## Learning Objectives

By the end of this tutorial, you will understand how to:
- Configure Strands agents to use AgentCore Browser Tool
- Implement CAPTCHA detection through AgentCore's secure browser sessions
- Leverage AgentCore's isolation and security for CAPTCHA handling
- Use AgentCore's observability features for debugging CAPTCHA workflows
- Build production-ready CAPTCHA solutions with enterprise-grade security

## 1. Strands Three-Layer Architecture for CAPTCHA Handling

### Architecture Overview

Strands implements a sophisticated three-layer architecture for CAPTCHA handling that separates concerns and maximizes both security and efficiency:

```
┌─────────────────────────────────────────────────────────────────┐
│                    STRANDS AGENT LAYER                         │
│                     (Orchestrator)                             │
│  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐ │
│  │ Decision Making │  │ Workflow Mgmt   │  │ Error Handling  │ │
│  │ & Planning      │  │ & Coordination  │  │ & Recovery      │ │
│  └─────────────────┘  └─────────────────┘  └─────────────────┘ │
└─────────────────────────────────────────────────────────────────┘
                                │
                                ▼
┌─────────────────────────────────────────────────────────────────┐
│                 AGENTCORE BROWSER LAYER                        │
│                   (Browser Executor)                           │
│  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐ │
│  │ Secure Browser  │  │ DOM Navigation  │  │ Screenshot      │ │
│  │ Sessions        │  │ & Interaction   │  │ Capture         │ │
│  └─────────────────┘  └─────────────────┘  └─────────────────┘ │
└─────────────────────────────────────────────────────────────────┘
                                │
                                ▼
┌─────────────────────────────────────────────────────────────────┐
│                   BEDROCK AI LAYER                             │
│                   (AI Analyzer)                                │
│  ┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐ │
│  │ Vision Models   │  │ Language Models │  │ Multi-Modal     │ │
│  │ (Claude Vision) │  │ (Claude Text)   │  │ Analysis        │ │
│  └─────────────────┘  └─────────────────┘  └─────────────────┘ │
└─────────────────────────────────────────────────────────────────┘
```

### Layer Responsibilities

#### 1. Strands Agent Layer (Orchestrator)
- **Decision Making**: Determines CAPTCHA handling strategy based on detection results
- **Workflow Coordination**: Orchestrates the entire CAPTCHA solving process
- **Service Integration**: Coordinates between AgentCore and Bedrock services
- **Error Recovery**: Implements retry logic and fallback strategies
- **State Management**: Maintains context across multi-step CAPTCHA processes

#### 2. AgentCore Browser Layer (Browser Executor)
- **Secure Execution**: Provides isolated, enterprise-grade browser sessions
- **DOM Interaction**: Handles all browser-based operations and element detection
- **Screenshot Capture**: Takes high-quality screenshots for AI analysis
- **Form Submission**: Submits CAPTCHA solutions back to target websites
- **Session Management**: Manages browser lifecycle and cleanup

#### 3. Bedrock AI Layer (AI Analyzer)
- **Vision Analysis**: Processes CAPTCHA images using Claude Vision models
- **Text Recognition**: Extracts and interprets text from CAPTCHA images
- **Pattern Recognition**: Identifies objects and patterns in image-based CAPTCHAs
- **Solution Generation**: Generates appropriate responses for different CAPTCHA types
- **Confidence Scoring**: Provides confidence metrics for solution quality

### Architecture Benefits

1. **Separation of Concerns**: Each layer has a specific responsibility
2. **Scalability**: Layers can scale independently based on demand
3. **Security**: AgentCore provides enterprise-grade isolation
4. **Flexibility**: Easy to swap or upgrade individual components
5. **Observability**: Each layer provides detailed logging and monitoring
6. **Reliability**: Built-in error handling and recovery at each layer

### Service Coordination Flow

Here's how Strands coordinates the three layers for CAPTCHA handling:

```mermaid
sequenceDiagram
    participant SA as Strands Agent
    participant AC as AgentCore Browser
    participant BR as Bedrock AI
    participant WS as Target Website

    Note over SA: 1. CAPTCHA Detection Phase
    SA->>AC: Create secure browser session
    AC->>WS: Navigate to target page
    AC->>SA: Page loaded successfully
    SA->>AC: Scan for CAPTCHA elements
    AC->>AC: DOM analysis & element detection
    AC->>SA: CAPTCHA detected (type, location)

    Note over SA: 2. CAPTCHA Analysis Phase
    SA->>AC: Capture CAPTCHA screenshot
    AC->>SA: Screenshot data (base64)
    SA->>BR: Analyze CAPTCHA image
    BR->>BR: Vision model processing
    BR->>SA: Solution + confidence score

    Note over SA: 3. Solution Submission Phase
    SA->>AC: Submit CAPTCHA solution
    AC->>WS: Fill form & submit
    WS->>AC: Response (success/failure)
    AC->>SA: Submission result
    SA->>SA: Update workflow state

    Note over SA: 4. Verification Phase
    SA->>AC: Verify CAPTCHA completion
    AC->>AC: Check page state changes
    AC->>SA: Verification result
    SA->>AC: Close browser session
```

In [ ]:
# Strands Agent Decision-Making Example
from strands import Agent, AgentConfig, ToolResult
from strands.workflows import WorkflowContext
from typing import Dict, Any

class CaptchaOrchestrationAgent(Agent):
    """Strands agent that orchestrates CAPTCHA handling across multiple services"""
    
    def __init__(self, config: AgentConfig):
        super().__init__(config)
        self.agentcore_client = None
        self.bedrock_client = None
        
    async def handle_captcha_workflow(self, page_url: str, task_description: str) -> Dict[str, Any]:
        """Orchestrate complete CAPTCHA handling workflow"""
        
        workflow_context = WorkflowContext({
            'page_url': page_url,
            'task': task_description,
            'start_time': time.time(),
            'attempts': 0,
            'max_attempts': 3
        })
        
        print(f"🎯 Starting CAPTCHA workflow for: {page_url}")
        print(f"📋 Task: {task_description}")
        
        try:
            # Phase 1: Detection and Analysis
            detection_result = await self._orchestrate_detection_phase(workflow_context)
            
            if not detection_result['captcha_found']:
                print("✅ No CAPTCHA detected - proceeding with original task")
                return await self._execute_original_task(workflow_context)
            
            # Phase 2: AI-Powered Solution
            solution_result = await self._orchestrate_solution_phase(workflow_context, detection_result)
            
            # Phase 3: Submission and Verification
            submission_result = await self._orchestrate_submission_phase(workflow_context, solution_result)
            
            # Phase 4: Task Completion
            if submission_result['success']:
                final_result = await self._execute_original_task(workflow_context)
                final_result['captcha_handled'] = True
                return final_result
            else:
                return self._handle_captcha_failure(workflow_context, submission_result)
                
        except Exception as e:
            print(f"❌ Workflow orchestration failed: {e}")
            return {
                'success': False,
                'error': str(e),
                'workflow_phase': workflow_context.get('current_phase', 'unknown')
            }
    
    async def _orchestrate_detection_phase(self, context: WorkflowContext) -> Dict[str, Any]:
        """Orchestrate CAPTCHA detection using AgentCore Browser Tool"""
        
        context.set('current_phase', 'detection')
        print("🔍 Phase 1: CAPTCHA Detection")
        
        # Decision: Choose detection strategy based on page characteristics
        detection_strategy = await self._choose_detection_strategy(context.get('page_url'))
        print(f"📊 Selected detection strategy: {detection_strategy}")
        
        # Coordinate with AgentCore for browser-based detection
        detection_tool = self.get_tool('captcha_detector')
        detection_result = await detection_tool.execute(
            page_url=context.get('page_url'),
            strategy=detection_strategy
        )
        
        if detection_result.success:
            print(f"✅ CAPTCHA detection completed: {detection_result.data['captcha_type']}")
            context.set('detection_data', detection_result.data)
            return detection_result.data
        else:
            print(f"❌ CAPTCHA detection failed: {detection_result.error}")
            raise Exception(f"Detection phase failed: {detection_result.error}")
    
    async def _orchestrate_solution_phase(self, context: WorkflowContext, detection_data: Dict[str, Any]) -> Dict[str, Any]:
        """Orchestrate CAPTCHA solution using Bedrock AI models"""
        
        context.set('current_phase', 'solution')
        print("🧠 Phase 2: AI-Powered CAPTCHA Solution")
        
        captcha_type = detection_data['captcha_type']
        
        # Decision: Choose AI model based on CAPTCHA type
        model_config = await self._choose_ai_model(captcha_type)
        print(f"🤖 Selected AI model: {model_config['model_name']}")
        
        # Coordinate with Bedrock for AI analysis
        solver_tool = self.get_tool('captcha_solver')
        solution_result = await solver_tool.execute(
            captcha_data=detection_data,
            model_config=model_config
        )
        
        if solution_result.success:
            confidence = solution_result.data.get('confidence', 0.0)
            print(f"✅ CAPTCHA solution generated (confidence: {confidence:.2f})")
            
            # Decision: Proceed only if confidence is above threshold
            if confidence >= 0.7:
                context.set('solution_data', solution_result.data)
                return solution_result.data
            else:
                print(f"⚠️ Low confidence solution ({confidence:.2f}) - requesting retry")
                return await self._retry_solution_with_different_model(context, detection_data)
        else:
            print(f"❌ CAPTCHA solution failed: {solution_result.error}")
            raise Exception(f"Solution phase failed: {solution_result.error}")
    
    async def _orchestrate_submission_phase(self, context: WorkflowContext, solution_data: Dict[str, Any]) -> Dict[str, Any]:
        """Orchestrate CAPTCHA solution submission using AgentCore Browser Tool"""
        
        context.set('current_phase', 'submission')
        print("📤 Phase 3: Solution Submission")
        
        # Decision: Choose submission method based on CAPTCHA type
        submission_method = await self._choose_submission_method(solution_data['captcha_type'])
        print(f"📋 Selected submission method: {submission_method}")
        
        # Coordinate with AgentCore for browser interaction
        submission_tool = self.get_tool('captcha_submitter')
        submission_result = await submission_tool.execute(
            solution_data=solution_data,
            method=submission_method,
            detection_data=context.get('detection_data')
        )
        
        if submission_result.success:
            print("✅ CAPTCHA solution submitted successfully")
            
            # Verify submission was accepted
            verification_result = await self._verify_captcha_completion(context)
            
            if verification_result['completed']:
                print("🎉 CAPTCHA successfully solved and verified!")
                return {'success': True, 'verified': True}
            else:
                print("⚠️ CAPTCHA submission not accepted - may need retry")
                return {'success': False, 'reason': 'submission_rejected'}
        else:
            print(f"❌ CAPTCHA submission failed: {submission_result.error}")
            return {'success': False, 'reason': 'submission_error', 'error': submission_result.error}
    
    async def _choose_detection_strategy(self, page_url: str) -> str:
        """Agent decision-making: Choose optimal detection strategy"""
        
        # Analyze page characteristics to choose strategy
        if 'google.com' in page_url:
            return 'recaptcha_focused'
        elif 'cloudflare' in page_url:
            return 'turnstile_focused'
        else:
            return 'comprehensive_scan'
    
    async def _choose_ai_model(self, captcha_type: str) -> Dict[str, Any]:
        """Agent decision-making: Choose optimal AI model for CAPTCHA type"""
        
        model_configs = {
            'text_captcha': {
                'model_name': 'anthropic.claude-3-sonnet-20240229-v1:0',
                'focus': 'text_recognition',
                'temperature': 0.1
            },
            'image_selection': {
                'model_name': 'anthropic.claude-3-sonnet-20240229-v1:0',
                'focus': 'object_detection',
                'temperature': 0.2
            },
            'recaptcha': {
                'model_name': 'anthropic.claude-3-opus-20240229-v1:0',
                'focus': 'complex_reasoning',
                'temperature': 0.1
            }
        }
        
        return model_configs.get(captcha_type, model_configs['text_captcha'])
    
    async def _choose_submission_method(self, captcha_type: str) -> str:
        """Agent decision-making: Choose optimal submission method"""
        
        submission_methods = {
            'text_captcha': 'direct_input',
            'recaptcha': 'checkbox_click',
            'image_selection': 'multi_click',
            'hcaptcha': 'iframe_interaction'
        }
        
        return submission_methods.get(captcha_type, 'direct_input')
    
    async def _retry_solution_with_different_model(self, context: WorkflowContext, detection_data: Dict[str, Any]) -> Dict[str, Any]:
        """Agent decision-making: Retry with different model configuration"""
        
        attempts = context.get('solution_attempts', 0) + 1
        context.set('solution_attempts', attempts)
        
        if attempts >= 2:
            raise Exception("Maximum solution attempts reached")
        
        print(f"🔄 Retrying with alternative model (attempt {attempts + 1})")
        
        # Use more powerful model for retry
        alternative_config = {
            'model_name': 'anthropic.claude-3-opus-20240229-v1:0',
            'focus': 'enhanced_analysis',
            'temperature': 0.05
        }
        
        solver_tool = self.get_tool('captcha_solver')
        retry_result = await solver_tool.execute(
            captcha_data=detection_data,
            model_config=alternative_config
        )
        
        if retry_result.success:
            return retry_result.data
        else:
            raise Exception(f"Retry solution failed: {retry_result.error}")
    
    async def _verify_captcha_completion(self, context: WorkflowContext) -> Dict[str, Any]:
        """Verify CAPTCHA was successfully completed"""
        
        verification_tool = self.get_tool('captcha_verifier')
        result = await verification_tool.execute(
            page_url=context.get('page_url'),
            detection_data=context.get('detection_data')
        )
        
        return result.data if result.success else {'completed': False}
    
    async def _execute_original_task(self, context: WorkflowContext) -> Dict[str, Any]:
        """Execute the original task after CAPTCHA is handled"""
        
        print(f"🎯 Executing original task: {context.get('task')}")
        
        # This would contain the actual task execution logic
        # For demonstration, we'll simulate task completion
        
        return {
            'success': True,
            'task_completed': True,
            'execution_time': time.time() - context.get('start_time'),
            'captcha_handled': context.get('detection_data', {}).get('captcha_found', False)
        }
    
    async def _handle_captcha_failure(self, context: WorkflowContext, submission_result: Dict[str, Any]) -> Dict[str, Any]:
        """Handle CAPTCHA failure with appropriate recovery strategy"""
        
        attempts = context.get('attempts', 0) + 1
        context.set('attempts', attempts)
        
        if attempts < context.get('max_attempts', 3):
            print(f"🔄 CAPTCHA failed - retrying (attempt {attempts + 1})")
            
            # Wait before retry (exponential backoff)
            await asyncio.sleep(2 ** attempts)
            
            # Retry the entire workflow
            return await self.handle_captcha_workflow(
                context.get('page_url'),
                context.get('task')
            )
        else:
            print("❌ Maximum CAPTCHA attempts reached - workflow failed")
            return {
                'success': False,
                'reason': 'max_attempts_reached',
                'attempts': attempts,
                'last_error': submission_result.get('error')
            }

print("✅ Strands CAPTCHA Orchestration Agent implemented!")
print("🎯 Demonstrates agent decision-making and service coordination patterns")

### Key Orchestration Patterns

The Strands agent demonstrates several important orchestration patterns:

#### 1. **Adaptive Strategy Selection**
```python
# Agent chooses detection strategy based on page characteristics
detection_strategy = await self._choose_detection_strategy(page_url)

# Agent selects AI model based on CAPTCHA complexity
model_config = await self._choose_ai_model(captcha_type)
```

#### 2. **Multi-Service Coordination**
```python
# Coordinate AgentCore for detection
detection_result = await detection_tool.execute(page_url, strategy=strategy)

# Coordinate Bedrock for AI analysis
solution_result = await solver_tool.execute(captcha_data, model_config)

# Coordinate AgentCore for submission
submission_result = await submission_tool.execute(solution_data, method)
```

#### 3. **Intelligent Error Recovery**
```python
# Retry with different model if confidence is low
if confidence < 0.7:
    return await self._retry_solution_with_different_model(context, detection_data)

# Exponential backoff for failed attempts
await asyncio.sleep(2 ** attempts)
```

#### 4. **State Management Across Services**
```python
# Maintain workflow context across all phases
workflow_context = WorkflowContext({
    'page_url': page_url,
    'detection_data': detection_result,
    'solution_data': solution_result,
    'attempts': attempt_count
})
```

This architecture ensures that Strands agents can handle complex CAPTCHA scenarios while maintaining enterprise-grade security, reliability, and observability across all three layers.

## 1.2 Advanced Workflow Coordination Examples

### Multi-Step CAPTCHA Workflow Coordination

Strands agents excel at coordinating complex, multi-step CAPTCHA processes that require state management across service boundaries. Here are detailed examples of how Strands orchestrates these workflows:

#### Example 1: Complex reCAPTCHA with Image Selection

This example shows how Strands coordinates a multi-step reCAPTCHA that requires both checkbox interaction and image selection:

```mermaid
sequenceDiagram
    participant SA as Strands Agent
    participant SM as State Manager
    participant AC as AgentCore Browser
    participant BR as Bedrock Vision
    participant WS as Website

    Note over SA: Multi-Step reCAPTCHA Workflow
    
    SA->>SM: Initialize workflow state
    SM->>SA: State context created
    
    SA->>AC: Create browser session
    AC->>WS: Navigate to page
    WS->>AC: Page loaded with reCAPTCHA
    
    SA->>AC: Click reCAPTCHA checkbox
    AC->>WS: Checkbox interaction
    WS->>AC: Image challenge appears
    AC->>SA: Challenge detected
    
    SA->>SM: Update state: image_challenge
    SA->>AC: Capture challenge screenshot
    AC->>SA: Screenshot data
    
    SA->>BR: Analyze image grid
    BR->>BR: Vision model processing
    BR->>SA: Selected images + confidence
    
    SA->>SM: Update state: solution_ready
    SA->>AC: Click selected images
    AC->>WS: Submit image selection
    
    WS->>AC: Verification result
    AC->>SA: Success/failure status
    SA->>SM: Update final state
```

## 1.1 Detailed Architecture Components

### Strands Agent Orchestration Layer

The Strands Agent acts as the intelligent orchestrator that coordinates all CAPTCHA handling activities. Here's how it manages the complete workflow:

```mermaid
graph TB
    subgraph "Strands Agent Orchestration Layer"
        DM[Decision Manager]
        WC[Workflow Coordinator]
        SM[State Manager]
        EH[Error Handler]
        PM[Performance Monitor]
    end
    
    subgraph "Service Coordination"
        AC[AgentCore Browser]
        BR[Bedrock AI Models]
        OB[Observability Layer]
    end
    
    subgraph "CAPTCHA Workflow States"
        S1[Detection]
        S2[Analysis]
        S3[Solution]
        S4[Submission]
        S5[Verification]
    end
    
    DM --> WC
    WC --> SM
    SM --> EH
    EH --> PM
    
    WC --> AC
    WC --> BR
    PM --> OB
    
    SM --> S1
    S1 --> S2
    S2 --> S3
    S3 --> S4
    S4 --> S5
    
    style DM fill:#e3f2fd
    style WC fill:#f3e5f5
    style SM fill:#e8f5e8
    style EH fill:#fff3e0
```

#### Decision Manager Responsibilities:
- **Strategy Selection**: Chooses optimal detection and solving strategies based on page characteristics
- **Model Selection**: Selects appropriate Bedrock AI models based on CAPTCHA complexity
- **Resource Allocation**: Determines AgentCore session configuration and scaling needs
- **Retry Logic**: Decides when and how to retry failed CAPTCHA attempts

#### Workflow Coordinator Responsibilities:
- **Service Orchestration**: Coordinates between AgentCore Browser and Bedrock AI services
- **Phase Management**: Manages transitions between detection, analysis, solution, and submission phases
- **Parallel Processing**: Coordinates concurrent operations when possible (e.g., multiple CAPTCHA analysis)
- **Dependency Management**: Ensures proper sequencing of dependent operations

#### State Manager Responsibilities:
- **Context Preservation**: Maintains workflow state across service boundaries
- **Session Tracking**: Tracks AgentCore browser sessions and their lifecycle
- **Progress Monitoring**: Monitors progress through multi-step CAPTCHA processes
- **Rollback Support**: Enables rollback to previous states on failure

### AgentCore Browser Executor Layer

AgentCore provides enterprise-grade browser automation with built-in security and isolation:

```mermaid
graph LR
    subgraph "AgentCore Browser Layer"
        SS[Secure Sessions]
        DI[DOM Interaction]
        SC[Screenshot Capture]
        FS[Form Submission]
        SM[Session Management]
    end
    
    subgraph "Security Features"
        ISO[VM Isolation]
        ENC[Data Encryption]
        AUD[Audit Logging]
        NET[Network Security]
    end
    
    subgraph "Observability"
        LV[Live Viewing]
        SR[Session Replay]
        CT[CloudTrail Logs]
        PM[Performance Metrics]
    end
    
    SS --> ISO
    DI --> ENC
    SC --> AUD
    FS --> NET
    SM --> LV
    
    ISO --> SR
    ENC --> CT
    AUD --> PM
    
    style SS fill:#e8f5e8
    style ISO fill:#ffebee
    style LV fill:#e3f2fd
```

#### AgentCore Security Benefits:
- **VM-Level Isolation**: Each browser session runs in an isolated virtual machine
- **Zero Data Persistence**: No data persists between sessions, ensuring privacy
- **Encrypted Communication**: All data transmission is encrypted in transit and at rest
- **Audit Trail**: Complete audit trail of all browser interactions

### Bedrock AI Analyzer Layer

Bedrock provides enterprise-grade AI models for CAPTCHA analysis:

```mermaid
graph TB
    subgraph "Bedrock AI Models"
        CV[Claude Vision]
        CT[Claude Text]
        MM[Multi-Modal Analysis]
    end
    
    subgraph "CAPTCHA Analysis Types"
        TR[Text Recognition]
        OD[Object Detection]
        PR[Pattern Recognition]
        CR[Complex Reasoning]
    end
    
    subgraph "Output Processing"
        SG[Solution Generation]
        CS[Confidence Scoring]
        QA[Quality Assessment]
    end
    
    CV --> TR
    CV --> OD
    CT --> PR
    MM --> CR
    
    TR --> SG
    OD --> CS
    PR --> QA
    CR --> SG
    
    style CV fill:#e3f2fd
    style CT fill:#f3e5f5
    style MM fill:#e8f5e8
```

#### AI Model Selection Strategy:
- **Text CAPTCHAs**: Claude Sonnet for optimal text recognition accuracy
- **Image Selection**: Claude Sonnet for object detection and classification
- **Complex CAPTCHAs**: Claude Opus for advanced reasoning and pattern recognition
- **Multi-Modal**: Combined vision and text models for hybrid CAPTCHAs

### Role Separation Benefits

The three-layer architecture provides clear separation of concerns:

| Layer | Primary Role | Key Benefits |
|-------|-------------|-------------|
| **Strands (Orchestrator)** | Decision-making and coordination | • Intelligent workflow management<br>• Cross-service state management<br>• Error recovery and retry logic<br>• Performance optimization |
| **AgentCore (Executor)** | Secure browser automation | • Enterprise-grade security<br>• VM-level isolation<br>• Built-in observability<br>• Serverless scaling |
| **Bedrock (Analyzer)** | AI-powered analysis | • State-of-the-art vision models<br>• Multi-modal capabilities<br>• Enterprise compliance<br>• Consistent performance |

This separation enables:
- **Independent Scaling**: Each layer scales based on its specific demands
- **Technology Evolution**: Layers can be upgraded independently
- **Security Boundaries**: Clear security boundaries between components
- **Specialized Optimization**: Each layer optimized for its specific function
- **Fault Isolation**: Failures in one layer don't cascade to others

### Enterprise Integration Patterns

The architecture supports enterprise integration patterns:

```mermaid
graph TB
    subgraph "Enterprise Integration"
        API[REST/GraphQL APIs]
        MSG[Message Queues]
        EVT[Event Streams]
        WEB[Webhooks]
    end
    
    subgraph "Strands CAPTCHA Service"
        SA[Strands Agent]
        AC[AgentCore Browser]
        BR[Bedrock AI]
    end
    
    subgraph "Enterprise Systems"
        CRM[CRM Systems]
        ERP[ERP Systems]
        WF[Workflow Engines]
        MON[Monitoring Systems]
    end
    
    API --> SA
    MSG --> SA
    EVT --> SA
    WEB --> SA
    
    SA --> AC
    SA --> BR
    
    SA --> CRM
    SA --> ERP
    SA --> WF
    SA --> MON
    
    style SA fill:#e3f2fd
    style API fill:#f3e5f5
    style CRM fill:#e8f5e8
```

This enables integration with:
- **Business Process Automation**: Embed CAPTCHA handling in business workflows
- **Customer Journey Orchestration**: Handle CAPTCHAs in customer-facing processes
- **Data Pipeline Integration**: Include CAPTCHA handling in data collection workflows
- **Monitoring and Alerting**: Enterprise-grade monitoring and alerting capabilities

In [ ]:
# Advanced Multi-Step CAPTCHA Workflow Coordination
from strands.workflows import AdvancedWorkflow, StateTransition, ConditionalStep
from strands.state import WorkflowStateManager, StateSnapshot
from typing import Dict, Any, List, Optional

class MultiStepCaptchaWorkflow(AdvancedWorkflow):
    """Advanced Strands workflow for multi-step CAPTCHA coordination"""
    
    def __init__(self):
        super().__init__()
        self.state_manager = WorkflowStateManager()
        self.setup_workflow_steps()
    
    def setup_workflow_steps(self):
        """Define the complete multi-step CAPTCHA workflow"""
        
        # Step 1: Initial Detection and State Setup
        self.add_step(ConditionalStep(
            name="initialize_captcha_detection",
            action=self.initialize_detection_phase,
            conditions={
                'page_loaded': True,
                'session_active': True
            },
            next_steps={
                'captcha_found': 'analyze_captcha_type',
                'no_captcha': 'execute_original_task',
                'error': 'handle_detection_error'
            }
        ))
        
        # Step 2: CAPTCHA Type Analysis and Strategy Selection
        self.add_step(ConditionalStep(
            name="analyze_captcha_type",
            action=self.analyze_and_plan_strategy,
            conditions={
                'captcha_detected': True
            },
            next_steps={
                'simple_text': 'handle_text_captcha',
                'recaptcha_checkbox': 'handle_recaptcha_workflow',
                'image_selection': 'handle_image_captcha',
                'complex_multi_step': 'handle_complex_workflow'
            }
        ))
        
        # Step 3: Multi-Step reCAPTCHA Handling
        self.add_step(ConditionalStep(
            name="handle_recaptcha_workflow",
            action=self.coordinate_recaptcha_workflow,
            conditions={
                'captcha_type': 'recaptcha'
            },
            next_steps={
                'checkbox_success': 'verify_completion',
                'image_challenge': 'handle_image_challenge',
                'audio_challenge': 'handle_audio_challenge',
                'failure': 'retry_with_different_strategy'
            }
        ))
        
        # Step 4: Image Challenge Coordination
        self.add_step(ConditionalStep(
            name="handle_image_challenge",
            action=self.coordinate_image_analysis,
            conditions={
                'image_challenge_active': True
            },
            next_steps={
                'solution_ready': 'submit_image_solution',
                'need_more_images': 'handle_additional_images',
                'low_confidence': 'retry_with_enhanced_model'
            }
        ))
    
    async def initialize_detection_phase(self, context: WorkflowContext) -> StateTransition:
        """Initialize CAPTCHA detection with comprehensive state management"""
        
        print("🚀 Initializing multi-step CAPTCHA detection workflow")
        
        # Create state snapshot for rollback capability
        initial_state = self.state_manager.create_snapshot(context)
        context.set('initial_state_snapshot', initial_state)
        
        # Initialize workflow tracking
        workflow_id = f"captcha_workflow_{int(time.time())}"
        context.set('workflow_id', workflow_id)
        context.set('start_time', time.time())
        context.set('step_history', [])
        
        print(f"📋 Workflow ID: {workflow_id}")
        
        try:
            # Coordinate with AgentCore for initial page analysis
            detection_tool = self.get_tool('advanced_captcha_detector')
            detection_result = await detection_tool.execute(
                page_url=context.get('page_url'),
                analysis_depth='comprehensive',
                state_tracking=True
            )
            
            if detection_result.success:
                # Update state with detection results
                context.set('captcha_detected', True)
                context.set('detection_data', detection_result.data)
                context.set('captcha_elements', detection_result.data.get('elements', []))
                
                # Track step completion
                self._track_step_completion(context, 'detection', detection_result.data)
                
                print(f"✅ CAPTCHA detected: {detection_result.data.get('captcha_type')}")
                print(f"📊 Complexity score: {detection_result.data.get('complexity_score', 0)}")
                
                return StateTransition('captcha_found', detection_result.data)
            else:
                context.set('captcha_detected', False)
                print("✅ No CAPTCHA detected - proceeding with original task")
                return StateTransition('no_captcha', {})
                
        except Exception as e:
            print(f"❌ Detection initialization failed: {e}")
            context.set('error', str(e))
            return StateTransition('error', {'error': str(e)})
    
    async def coordinate_recaptcha_workflow(self, context: WorkflowContext) -> StateTransition:
        """Coordinate complex reCAPTCHA workflow with state management"""
        
        print("🔄 Coordinating multi-step reCAPTCHA workflow")
        
        strategy = context.get('coordination_strategy', {})
        detection_data = context.get('detection_data', {})
        
        # Phase 1: Initial checkbox interaction
        print("📋 Phase 1: reCAPTCHA checkbox interaction")
        
        checkbox_result = await self._coordinate_checkbox_interaction(context, detection_data)
        
        if not checkbox_result['success']:
            return StateTransition('failure', checkbox_result)
        
        # Update state after checkbox interaction
        context.set('checkbox_completed', True)
        context.set('checkbox_result', checkbox_result)
        
        # Check if additional challenge appeared
        if checkbox_result.get('additional_challenge'):
            challenge_type = checkbox_result.get('challenge_type')
            
            print(f"🎯 Additional challenge detected: {challenge_type}")
            
            # Update state for challenge handling
            context.set('challenge_active', True)
            context.set('challenge_type', challenge_type)
            context.set('challenge_data', checkbox_result.get('challenge_data', {}))
            
            # Track challenge detection
            self._track_step_completion(context, 'challenge_detected', {
                'type': challenge_type,
                'data': checkbox_result.get('challenge_data', {})
            })
            
            if challenge_type == 'image_selection':
                return StateTransition('image_challenge', checkbox_result)
            elif challenge_type == 'audio':
                return StateTransition('audio_challenge', checkbox_result)
            else:
                return StateTransition('unknown_challenge', checkbox_result)
        else:
            print("✅ reCAPTCHA completed with checkbox only")
            self._track_step_completion(context, 'recaptcha_completed', checkbox_result)
            return StateTransition('checkbox_success', checkbox_result)
    
    async def coordinate_image_analysis(self, context: WorkflowContext) -> StateTransition:
        """Coordinate image analysis with Bedrock AI models and state tracking"""
        
        print("🖼️ Coordinating image challenge analysis")
        
        challenge_data = context.get('challenge_data', {})
        strategy = context.get('coordination_strategy', {})
        
        # Capture high-quality screenshot for analysis
        screenshot_result = await self._coordinate_screenshot_capture(context, challenge_data)
        
        if not screenshot_result['success']:
            return StateTransition('screenshot_failed', screenshot_result)
        
        # Update state with screenshot data
        context.set('screenshot_captured', True)
        context.set('screenshot_data', screenshot_result['data'])
        
        # Coordinate with Bedrock for AI analysis
        print("🤖 Coordinating Bedrock AI analysis")
        
        analysis_config = strategy.get('ai_config', {})
        analysis_result = await self._coordinate_ai_analysis(
            context, 
            screenshot_result['data'], 
            analysis_config
        )
        
        if analysis_result['success']:
            confidence = analysis_result.get('confidence', 0.0)
            solution = analysis_result.get('solution', {})
            
            print(f"✅ AI analysis completed (confidence: {confidence:.2f})")
            
            # Update state with analysis results
            context.set('ai_analysis_completed', True)
            context.set('solution_confidence', confidence)
            context.set('solution_data', solution)
            
            # Track analysis completion
            self._track_step_completion(context, 'ai_analysis', {
                'confidence': confidence,
                'solution_type': solution.get('type'),
                'elements_identified': len(solution.get('selected_elements', []))
            })
            
            # Decision: Proceed based on confidence level
            if confidence >= 0.8:
                print("🎯 High confidence solution - proceeding with submission")
                return StateTransition('solution_ready', analysis_result)
            elif confidence >= 0.6:
                print("⚠️ Medium confidence - may need verification")
                return StateTransition('solution_ready', analysis_result)
            else:
                print("❌ Low confidence solution - requesting enhanced analysis")
                return StateTransition('low_confidence', analysis_result)
        else:
            print(f"❌ AI analysis failed: {analysis_result.get('error')}")
            return StateTransition('analysis_failed', analysis_result)
    
    def _track_step_completion(self, context: WorkflowContext, step_name: str, step_data: Dict[str, Any]):
        """Track workflow step completion for state management"""
        
        step_history = context.get('step_history', [])
        
        step_record = {
            'step_name': step_name,
            'timestamp': time.time(),
            'data': step_data,
            'duration': time.time() - context.get('start_time', time.time())
        }
        
        step_history.append(step_record)
        context.set('step_history', step_history)
        context.set('current_step', step_name)
        
        print(f"📊 Step completed: {step_name} (duration: {step_record['duration']:.2f}s)")

print("✅ Advanced Multi-Step CAPTCHA Workflow implemented!")
print("🎯 Demonstrates comprehensive Strands coordination patterns")
print("📊 Includes state management across AgentCore and Bedrock services")

### State Management Across Service Boundaries

One of Strands' key strengths is maintaining consistent state across multiple services during complex CAPTCHA workflows:

#### Workflow State Tracking Example

```python
# State management across AgentCore and Bedrock operations
workflow_state = {
    'workflow_id': 'captcha_workflow_1234567890',
    'current_phase': 'image_analysis',
    'agentcore_session': {
        'session_id': 'ac_session_abc123',
        'browser_state': 'active',
        'page_url': 'https://example.com/form',
        'captcha_element': '#recaptcha-challenge'
    },
    'bedrock_analysis': {
        'model_used': 'claude-3-sonnet',
        'analysis_id': 'br_analysis_xyz789',
        'confidence_score': 0.87,
        'processing_time': 2.3
    },
    'step_history': [
        {'step': 'detection', 'timestamp': 1234567890, 'success': True},
        {'step': 'checkbox_click', 'timestamp': 1234567892, 'success': True},
        {'step': 'challenge_appeared', 'timestamp': 1234567893, 'success': True}
    ],
    'retry_count': 0,
    'max_retries': 3
}
```

#### Cross-Service Coordination Patterns

Strands implements several patterns for coordinating between AgentCore and Bedrock:

1. **Sequential Coordination**: Steps execute in sequence with state handoff
2. **Parallel Coordination**: Multiple services work simultaneously on different aspects
3. **Conditional Coordination**: Service selection based on runtime conditions
4. **Retry Coordination**: Intelligent retry with different service configurations

#### Example: Parallel Processing Coordination

```python
# Strands can coordinate parallel operations across services
async def coordinate_parallel_captcha_analysis(self, context: WorkflowContext):
    """Coordinate parallel CAPTCHA analysis for faster processing"""
    
    # Start multiple analysis approaches in parallel
    tasks = [
        # AgentCore: Capture multiple screenshot angles
        self._capture_multiple_screenshots(context),
        
        # Bedrock: Pre-load multiple AI models
        self._prepare_multiple_ai_models(context),
        
        # State Manager: Prepare rollback points
        self._create_state_snapshots(context)
    ]
    
    # Wait for all parallel operations to complete
    results = await asyncio.gather(*tasks, return_exceptions=True)
    
    # Coordinate results and choose best approach
    optimal_approach = self._select_optimal_approach(results)
    
    return optimal_approach
```

This coordination approach enables Strands to:
- **Minimize Latency**: Parallel operations reduce total processing time
- **Maximize Success Rate**: Multiple approaches increase solution probability
- **Optimize Resource Usage**: Efficient allocation across AgentCore and Bedrock
- **Maintain State Consistency**: Coordinated state management across all services

## 2. AgentCore Browser Tool Setup

Now let's configure Strands to work with Amazon Bedrock's AgentCore Browser Tool for secure CAPTCHA handling.

In [ ]:
# Install required packages (run this cell first)
!pip install -r requirements.txt

In [ ]:
# Import Strands framework components
from strands import Agent, Tool, ToolResult, AgentConfig
from strands.tools import ToolRegistry
from strands.workflows import Workflow, WorkflowStep, WorkflowContext
from strands.integrations.bedrock import BedrockAgentCore
from strands.browser import AgentCoreBrowserSession

# Import AWS Bedrock AgentCore components
import boto3
from botocore.exceptions import ClientError
from bedrock_agentcore import AgentCoreBrowserClient, BrowserSession

# Import standard libraries
import asyncio
import json
import base64
import time
from datetime import datetime
from typing import Dict, Any, Optional, List

# Import image processing for CAPTCHA analysis
from PIL import Image
import numpy as np

print("✅ Strands + AgentCore Browser Tool dependencies imported!")
print("🔒 Ready for secure, serverless CAPTCHA handling!")

## 2. AgentCore Browser Tool Configuration

Configure the AgentCore Browser Tool client for secure, isolated browser sessions.

In [ ]:
# Configure AgentCore Browser Tool client
def setup_agentcore_browser():
    """Initialize AgentCore Browser Tool for secure browser sessions"""
    try:
        # Initialize AgentCore Browser client
        agentcore_client = AgentCoreBrowserClient(
            region_name='us-east-1',  # Adjust region as needed
            session_config={
                'isolation_level': 'high',  # Enterprise-grade isolation
                'observability': True,      # Enable session replay and logging
                'auto_scaling': True,       # Enable automatic scaling
                'session_timeout': 300      # 5 minute timeout
            }
        )
        
        # Test connection and get service status
        status = agentcore_client.get_service_status()
        print("✅ AgentCore Browser Tool configured successfully!")
        print(f"🔒 Isolation level: {status.get('isolation_level', 'standard')}")
        print(f"📊 Available browser capacity: {status.get('available_sessions', 'unlimited')}")
        print(f"🔍 Observability enabled: {status.get('observability_enabled', False)}")
        
        return agentcore_client
    except Exception as e:
        print(f"❌ Error configuring AgentCore Browser Tool: {e}")
        print("💡 Ensure AWS credentials and AgentCore permissions are configured")
        return None

# Initialize AgentCore Browser client
agentcore_browser = setup_agentcore_browser()

# Also set up Bedrock for vision models
bedrock_client = boto3.client('bedrock-runtime', region_name='us-east-1')

## 3. CAPTCHA Detection Tool Implementation

Implement Strands Tool for CAPTCHA detection using AgentCore Browser Tool.

In [ ]:
class CaptchaDetectionTool(Tool):
    """Strands tool for detecting CAPTCHAs using AgentCore Browser Tool"""
    
    name = "captcha_detector"
    description = "Detects various types of CAPTCHAs on web pages including reCAPTCHA, hCaptcha, and image-based CAPTCHAs using AgentCore Browser Tool"
    
    def __init__(self, agentcore_client):
        super().__init__()
        self.agentcore = agentcore_client
        self.session = None
    
    async def execute(self, page_url: str, **kwargs) -> ToolResult:
        """Execute CAPTCHA detection on the specified page using AgentCore"""
        try:
            print(f"🔍 Starting CAPTCHA detection for: {page_url}")
            
            # Create isolated AgentCore browser session
            self.session = await self._create_secure_session()
            
            # Navigate to the page
            await self.session.navigate(page_url)
            print(f"✅ Navigated to {page_url} in isolated session")
            
            # Detect CAPTCHA elements
            detection_result = await self._detect_captcha_elements()
            
            # Capture screenshot for analysis
            screenshot_data = await self._capture_captcha_screenshot(detection_result)
            
            result_data = {
                'captcha_found': detection_result['found'],
                'captcha_type': detection_result.get('type'),
                'element_selector': detection_result.get('selector'),
                'confidence_score': detection_result.get('confidence', 0.0),
                'screenshot_data': screenshot_data,
                'page_url': page_url,
                'session_id': self.session.session_id,
                'detection_timestamp': datetime.now().isoformat()
            }
            
            return ToolResult(
                success=True,
                data=result_data,
                message=f"CAPTCHA detection completed for {page_url}. Found: {detection_result['found']}"
            )
            
        except Exception as e:
            print(f"❌ CAPTCHA detection failed: {e}")
            return ToolResult(
                success=False,
                error=str(e),
                message=f"CAPTCHA detection failed: {e}"
            )
        finally:
            # Clean up session
            if self.session:
                await self.session.close()
    
    async def _create_secure_session(self):
        """Create a secure, isolated AgentCore browser session"""
        session_config = {
            'isolation_level': 'high',
            'observability': True,
            'session_recording': True,
            'timeout': 300
        }
        
        session = await self.agentcore.create_session(session_config)
        print(f"🔒 Created secure session: {session.session_id}")
        return session
    
    async def _detect_captcha_elements(self) -> Dict[str, Any]:
        """Detect CAPTCHA elements using DOM analysis"""
        
        # Define CAPTCHA detection patterns
        captcha_selectors = {
            'recaptcha': [
                '.g-recaptcha',
                '[data-sitekey]',
                'iframe[src*="recaptcha"]',
                '#recaptcha',
                '.recaptcha-checkbox'
            ],
            'hcaptcha': [
                '.h-captcha',
                '[data-hcaptcha-sitekey]',
                'iframe[src*="hcaptcha"]',
                '#hcaptcha'
            ],
            'image_captcha': [
                'img[alt*="captcha" i]',
                'img[src*="captcha" i]',
                '.captcha-image',
                '#captcha-image'
            ],
            'text_captcha': [
                'input[name*="captcha" i]',
                'input[placeholder*="captcha" i]',
                '.captcha-input',
                '#captcha-input'
            ]
        }
        
        detection_result = {
            'found': False,
            'type': None,
            'selector': None,
            'confidence': 0.0
        }
        
        # Check each CAPTCHA type
        for captcha_type, selectors in captcha_selectors.items():
            for selector in selectors:
                try:
                    elements = await self.session.find_elements(selector)
                    if elements:
                        # Calculate confidence based on element visibility and attributes
                        confidence = await self._calculate_confidence(elements[0], captcha_type)
                        
                        if confidence > detection_result['confidence']:
                            detection_result.update({
                                'found': True,
                                'type': captcha_type,
                                'selector': selector,
                                'confidence': confidence
                            })
                            
                            print(f"✅ Detected {captcha_type} with confidence {confidence:.2f}")
                            
                except Exception as e:
                    print(f"⚠️ Error checking selector {selector}: {e}")
                    continue
        
        return detection_result
    
    async def _calculate_confidence(self, element, captcha_type: str) -> float:
        """Calculate confidence score for CAPTCHA detection"""
        confidence = 0.5  # Base confidence
        
        try:
            # Check if element is visible
            is_visible = await element.is_visible()
            if is_visible:
                confidence += 0.3
            
            # Check element attributes for CAPTCHA indicators
            attributes = await element.get_attributes()
            
            # Type-specific confidence adjustments
            if captcha_type == 'recaptcha':
                if 'data-sitekey' in attributes:
                    confidence += 0.2
                if 'recaptcha' in attributes.get('class', '').lower():
                    confidence += 0.1
            
            elif captcha_type == 'hcaptcha':
                if 'data-hcaptcha-sitekey' in attributes:
                    confidence += 0.2
                if 'hcaptcha' in attributes.get('class', '').lower():
                    confidence += 0.1
            
            # Check for common CAPTCHA keywords
            text_content = await element.get_text()
            captcha_keywords = ['captcha', 'verify', 'robot', 'human', 'security']
            
            for keyword in captcha_keywords:
                if keyword.lower() in text_content.lower():
                    confidence += 0.05
            
        except Exception as e:
            print(f"⚠️ Error calculating confidence: {e}")
        
        return min(confidence, 1.0)  # Cap at 1.0
    
    async def _capture_captcha_screenshot(self, detection_result: Dict[str, Any]) -> Optional[str]:
        """Capture screenshot of CAPTCHA area for analysis"""
        try:
            if detection_result['found'] and detection_result['selector']:
                # Take element screenshot
                element = await self.session.find_element(detection_result['selector'])
                screenshot_bytes = await element.screenshot()
                
                # Convert to base64 for storage/transmission
                screenshot_b64 = base64.b64encode(screenshot_bytes).decode('utf-8')
                
                print(f"📸 Captured CAPTCHA screenshot ({len(screenshot_bytes)} bytes)")
                return screenshot_b64
            else:
                # Take full page screenshot if no specific CAPTCHA found
                screenshot_bytes = await self.session.screenshot()
                screenshot_b64 = base64.b64encode(screenshot_bytes).decode('utf-8')
                
                print(f"📸 Captured full page screenshot ({len(screenshot_bytes)} bytes)")
                return screenshot_b64
                
        except Exception as e:
            print(f"⚠️ Error capturing screenshot: {e}")
            return None

print("✅ CaptchaDetectionTool implemented with AgentCore Browser Tool integration!")

## 4. DOM Element Detection Methods

Advanced DOM analysis methods for comprehensive CAPTCHA detection.

In [ ]:
class AdvancedCaptchaDetector:
    """Advanced CAPTCHA detection using multiple DOM analysis techniques"""
    
    def __init__(self, session):
        self.session = session
        
    async def comprehensive_captcha_scan(self) -> Dict[str, Any]:
        """Perform comprehensive CAPTCHA detection using multiple methods"""
        
        detection_methods = [
            self._detect_by_selectors,
            self._detect_by_iframe_analysis,
            self._detect_by_script_analysis,
            self._detect_by_visual_patterns,
            self._detect_by_network_requests
        ]
        
        all_results = []
        
        for method in detection_methods:
            try:
                result = await method()
                if result['found']:
                    all_results.append(result)
                    print(f"✅ {method.__name__}: Found {result['type']} (confidence: {result['confidence']:.2f})")
            except Exception as e:
                print(f"⚠️ {method.__name__} failed: {e}")
        
        # Combine results and return highest confidence detection
        if all_results:
            best_result = max(all_results, key=lambda x: x['confidence'])
            best_result['all_detections'] = all_results
            return best_result
        
        return {'found': False, 'confidence': 0.0, 'all_detections': []}
    
    async def _detect_by_selectors(self) -> Dict[str, Any]:
        """Detect CAPTCHAs using CSS selectors"""
        
        # Extended selector patterns
        selector_patterns = {
            'recaptcha_v2': [
                '.g-recaptcha',
                '[data-sitekey]',
                'div[class*="recaptcha"]',
                '#g-recaptcha-response'
            ],
            'recaptcha_v3': [
                'script[src*="recaptcha/releases/"]',
                '[data-action]',
                'input[name="g-recaptcha-response"]'
            ],
            'hcaptcha': [
                '.h-captcha',
                '[data-hcaptcha-sitekey]',
                'div[class*="hcaptcha"]',
                'textarea[name="h-captcha-response"]'
            ],
            'cloudflare_turnstile': [
                '.cf-turnstile',
                '[data-cf-turnstile-sitekey]',
                'div[class*="turnstile"]'
            ],
            'custom_image': [
                'img[alt*="captcha" i]',
                'img[src*="captcha" i]',
                'canvas[id*="captcha" i]',
                '.captcha-container img'
            ]
        }
        
        for captcha_type, selectors in selector_patterns.items():
            for selector in selectors:
                elements = await self.session.find_elements(selector)
                if elements:
                    confidence = await self._calculate_selector_confidence(elements[0], captcha_type)
                    if confidence > 0.7:  # High confidence threshold
                        return {
                            'found': True,
                            'type': captcha_type,
                            'method': 'selector_detection',
                            'selector': selector,
                            'confidence': confidence,
                            'element_count': len(elements)
                        }
        
        return {'found': False, 'confidence': 0.0}
    
    async def _detect_by_iframe_analysis(self) -> Dict[str, Any]:
        """Detect CAPTCHAs by analyzing iframes"""
        
        iframe_patterns = {
            'recaptcha': ['recaptcha', 'google.com/recaptcha'],
            'hcaptcha': ['hcaptcha', 'hcaptcha.com'],
            'cloudflare': ['challenges.cloudflare.com', 'turnstile']
        }
        
        iframes = await self.session.find_elements('iframe')
        
        for iframe in iframes:
            try:
                src = await iframe.get_attribute('src')
                if src:
                    for captcha_type, patterns in iframe_patterns.items():
                        for pattern in patterns:
                            if pattern.lower() in src.lower():
                                return {
                                    'found': True,
                                    'type': f'{captcha_type}_iframe',
                                    'method': 'iframe_analysis',
                                    'iframe_src': src,
                                    'confidence': 0.9
                                }
            except Exception as e:
                continue
        
        return {'found': False, 'confidence': 0.0}
    
    async def _detect_by_script_analysis(self) -> Dict[str, Any]:
        """Detect CAPTCHAs by analyzing loaded scripts"""
        
        script_patterns = {
            'recaptcha': ['www.google.com/recaptcha', 'recaptcha/releases'],
            'hcaptcha': ['hcaptcha.com/1/api.js', 'js.hcaptcha.com'],
            'cloudflare': ['challenges.cloudflare.com', 'turnstile']
        }
        
        scripts = await self.session.find_elements('script[src]')
        
        for script in scripts:
            try:
                src = await script.get_attribute('src')
                if src:
                    for captcha_type, patterns in script_patterns.items():
                        for pattern in patterns:
                            if pattern.lower() in src.lower():
                                return {
                                    'found': True,
                                    'type': f'{captcha_type}_script',
                                    'method': 'script_analysis',
                                    'script_src': src,
                                    'confidence': 0.85
                                }
            except Exception as e:
                continue
        
        return {'found': False, 'confidence': 0.0}
    
    async def _detect_by_visual_patterns(self) -> Dict[str, Any]:
        """Detect CAPTCHAs by analyzing visual patterns"""
        
        # Look for common CAPTCHA visual indicators
        visual_indicators = [
            'canvas',  # Often used for drawing CAPTCHAs
            'img[width][height]',  # CAPTCHA images often have specific dimensions
            'div[style*="cursor: pointer"]'  # Interactive CAPTCHA elements
        ]
        
        for selector in visual_indicators:
            elements = await self.session.find_elements(selector)
            
            for element in elements:
                try:
                    # Check element dimensions and position
                    rect = await element.get_bounding_rect()
                    
                    # Common CAPTCHA dimensions (approximate)
                    if (200 <= rect['width'] <= 400 and 
                        50 <= rect['height'] <= 200):
                        
                        # Additional checks for CAPTCHA-like content
                        parent_text = await self._get_surrounding_text(element)
                        
                        captcha_keywords = ['verify', 'human', 'robot', 'security', 'captcha']
                        keyword_matches = sum(1 for keyword in captcha_keywords 
                                            if keyword.lower() in parent_text.lower())
                        
                        if keyword_matches >= 2:
                            return {
                                'found': True,
                                'type': 'visual_captcha',
                                'method': 'visual_pattern_analysis',
                                'dimensions': rect,
                                'confidence': 0.7 + (keyword_matches * 0.1)
                            }
                            
                except Exception as e:
                    continue
        
        return {'found': False, 'confidence': 0.0}
    
    async def _detect_by_network_requests(self) -> Dict[str, Any]:
        """Detect CAPTCHAs by analyzing network requests"""
        
        # This would require network monitoring capabilities
        # For now, we'll simulate based on common CAPTCHA domains
        
        captcha_domains = [
            'www.google.com/recaptcha',
            'hcaptcha.com',
            'challenges.cloudflare.com',
            'funcaptcha.com'
        ]
        
        # In a real implementation, you would:
        # 1. Monitor network requests during page load
        # 2. Check for requests to known CAPTCHA services
        # 3. Analyze response headers and content types
        
        # For demonstration, we'll check if any scripts reference these domains
        page_source = await self.session.get_page_source()
        
        for domain in captcha_domains:
            if domain in page_source:
                captcha_type = domain.split('.')[1] if '.' in domain else 'unknown'
                return {
                    'found': True,
                    'type': f'{captcha_type}_network',
                    'method': 'network_analysis',
                    'domain': domain,
                    'confidence': 0.8
                }
        
        return {'found': False, 'confidence': 0.0}
    
    async def _calculate_selector_confidence(self, element, captcha_type: str) -> float:
        """Calculate confidence for selector-based detection"""
        confidence = 0.5
        
        try:
            # Check visibility
            if await element.is_visible():
                confidence += 0.2
            
            # Check attributes
            attributes = await element.get_attributes()
            
            # Type-specific attribute checks
            if captcha_type.startswith('recaptcha'):
                if 'data-sitekey' in attributes:
                    confidence += 0.3
            elif captcha_type.startswith('hcaptcha'):
                if 'data-hcaptcha-sitekey' in attributes:
                    confidence += 0.3
            
            # Check for CAPTCHA-related classes or IDs
            class_name = attributes.get('class', '')
            element_id = attributes.get('id', '')
            
            captcha_indicators = ['captcha', 'recaptcha', 'hcaptcha', 'turnstile']
            for indicator in captcha_indicators:
                if (indicator in class_name.lower() or 
                    indicator in element_id.lower()):
                    confidence += 0.1
            
        except Exception as e:
            print(f"⚠️ Error in confidence calculation: {e}")
        
        return min(confidence, 1.0)
    
    async def _get_surrounding_text(self, element) -> str:
        """Get text content around an element for context analysis"""
        try:
            # Get parent element text
            parent = await element.get_parent()
            if parent:
                return await parent.get_text()
            return await element.get_text()
        except Exception:
            return ""

print("✅ AdvancedCaptchaDetector implemented with comprehensive DOM analysis!")

## 5. Screenshot Capture Functionality

Implement screenshot capture for CAPTCHA analysis using AgentCore Browser Tool.

In [ ]:
class CaptchaScreenshotCapture:
    """Advanced screenshot capture functionality for CAPTCHA analysis"""
    
    def __init__(self, session):
        self.session = session
    
    async def capture_captcha_screenshots(self, detection_result: Dict[str, Any]) -> Dict[str, Any]:
        """Capture multiple types of screenshots for comprehensive CAPTCHA analysis"""
        
        screenshots = {
            'full_page': None,
            'captcha_element': None,
            'captcha_context': None,
            'viewport': None
        }
        
        try:
            # 1. Full page screenshot
            print("📸 Capturing full page screenshot...")
            full_page_bytes = await self.session.screenshot(full_page=True)
            screenshots['full_page'] = {
                'data': base64.b64encode(full_page_bytes).decode('utf-8'),
                'size': len(full_page_bytes),
                'type': 'full_page'
            }
            
            # 2. Viewport screenshot
            print("📸 Capturing viewport screenshot...")
            viewport_bytes = await self.session.screenshot(full_page=False)
            screenshots['viewport'] = {
                'data': base64.b64encode(viewport_bytes).decode('utf-8'),
                'size': len(viewport_bytes),
                'type': 'viewport'
            }
            
            # 3. CAPTCHA element screenshot (if detected)
            if detection_result.get('found') and detection_result.get('selector'):
                print(f"📸 Capturing CAPTCHA element screenshot for: {detection_result['selector']}")
                
                try:
                    element = await self.session.find_element(detection_result['selector'])
                    if element:
                        element_bytes = await element.screenshot()
                        screenshots['captcha_element'] = {
                            'data': base64.b64encode(element_bytes).decode('utf-8'),
                            'size': len(element_bytes),
                            'type': 'captcha_element',
                            'selector': detection_result['selector']
                        }
                        
                        # 4. CAPTCHA context screenshot (element + surrounding area)
                        context_bytes = await self._capture_element_context(element)
                        if context_bytes:
                            screenshots['captcha_context'] = {
                                'data': base64.b64encode(context_bytes).decode('utf-8'),
                                'size': len(context_bytes),
                                'type': 'captcha_context'
                            }
                            
                except Exception as e:
                    print(f"⚠️ Error capturing CAPTCHA element screenshot: {e}")
            
            # Add metadata
            screenshots['metadata'] = {
                'timestamp': datetime.now().isoformat(),
                'page_url': await self.session.get_current_url(),
                'viewport_size': await self.session.get_viewport_size(),
                'session_id': self.session.session_id,
                'total_screenshots': sum(1 for s in screenshots.values() if s is not None and isinstance(s, dict))
            }
            
            print(f"✅ Captured {screenshots['metadata']['total_screenshots']} screenshots successfully")
            
        except Exception as e:
            print(f"❌ Error in screenshot capture: {e}")
            screenshots['error'] = str(e)
        
        return screenshots
    
    async def _capture_element_context(self, element, padding: int = 50) -> Optional[bytes]:
        """Capture screenshot of element with surrounding context"""
        try:
            # Get element position and size
            rect = await element.get_bounding_rect()
            
            # Calculate context area with padding
            context_rect = {
                'x': max(0, rect['x'] - padding),
                'y': max(0, rect['y'] - padding),
                'width': rect['width'] + (2 * padding),
                'height': rect['height'] + (2 * padding)
            }
            
            # Capture screenshot of the context area
            context_bytes = await self.session.screenshot_region(context_rect)
            
            print(f"📸 Captured context screenshot: {context_rect['width']}x{context_rect['height']} pixels")
            return context_bytes
            
        except Exception as e:
            print(f"⚠️ Error capturing element context: {e}")
            return None
    
    async def save_screenshots_locally(self, screenshots: Dict[str, Any], base_filename: str = "captcha") -> Dict[str, str]:
        """Save screenshots to local files for analysis"""
        
        saved_files = {}
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        for screenshot_type, screenshot_data in screenshots.items():
            if isinstance(screenshot_data, dict) and 'data' in screenshot_data:
                try:
                    # Decode base64 data
                    image_bytes = base64.b64decode(screenshot_data['data'])
                    
                    # Create filename
                    filename = f"{base_filename}_{screenshot_type}_{timestamp}.png"
                    
                    # Save to file
                    with open(filename, 'wb') as f:
                        f.write(image_bytes)
                    
                    saved_files[screenshot_type] = filename
                    print(f"💾 Saved {screenshot_type} screenshot: {filename} ({screenshot_data['size']} bytes)")
                    
                except Exception as e:
                    print(f"❌ Error saving {screenshot_type} screenshot: {e}")
        
        return saved_files
    
    async def analyze_screenshot_quality(self, screenshot_data: Dict[str, Any]) -> Dict[str, Any]:
        """Analyze screenshot quality for CAPTCHA detection"""
        
        quality_metrics = {
            'resolution': None,
            'file_size': None,
            'clarity_score': None,
            'contrast_score': None,
            'suitability': 'unknown'
        }
        
        try:
            if 'data' in screenshot_data:
                # Decode image
                image_bytes = base64.b64decode(screenshot_data['data'])
                
                # Load with PIL for analysis
                from io import BytesIO
                image = Image.open(BytesIO(image_bytes))
                
                # Basic metrics
                quality_metrics['resolution'] = f"{image.width}x{image.height}"
                quality_metrics['file_size'] = len(image_bytes)
                
                # Convert to numpy for analysis
                img_array = np.array(image.convert('L'))  # Convert to grayscale
                
                # Calculate clarity (using Laplacian variance)
                # Higher variance indicates sharper image
                laplacian_var = np.var(np.gradient(img_array))
                quality_metrics['clarity_score'] = float(laplacian_var)
                
                # Calculate contrast (standard deviation of pixel values)
                contrast = np.std(img_array)
                quality_metrics['contrast_score'] = float(contrast)
                
                # Determine suitability for CAPTCHA analysis
                if (image.width >= 200 and image.height >= 100 and 
                    laplacian_var > 100 and contrast > 20):
                    quality_metrics['suitability'] = 'excellent'
                elif (image.width >= 150 and image.height >= 75 and 
                      laplacian_var > 50 and contrast > 15):
                    quality_metrics['suitability'] = 'good'
                elif (image.width >= 100 and image.height >= 50):
                    quality_metrics['suitability'] = 'fair'
                else:
                    quality_metrics['suitability'] = 'poor'
                
                print(f"📊 Screenshot quality: {quality_metrics['suitability']} ")
                print(f"   Resolution: {quality_metrics['resolution']}")
                print(f"   Clarity: {quality_metrics['clarity_score']:.2f}")
                print(f"   Contrast: {quality_metrics['contrast_score']:.2f}")
                
        except Exception as e:
            print(f"⚠️ Error analyzing screenshot quality: {e}")
            quality_metrics['error'] = str(e)
        
        return quality_metrics

print("✅ CaptchaScreenshotCapture implemented with advanced analysis capabilities!")

## 6. Demo: Basic CAPTCHA Detection

Demonstrate the basic CAPTCHA detection utilities in action.

In [ ]:
async def demo_basic_captcha_detection():
    """Demonstrate basic CAPTCHA detection functionality"""
    
    print("🎯 BASIC CAPTCHA DETECTION DEMO")
    print("=" * 50)
    
    # Test URLs with different CAPTCHA types
    test_urls = [
        {
            'url': 'https://www.google.com/recaptcha/api2/demo',
            'expected_type': 'recaptcha',
            'description': 'Google reCAPTCHA v2 Demo'
        },
        {
            'url': 'https://accounts.hcaptcha.com/demo',
            'expected_type': 'hcaptcha',
            'description': 'hCaptcha Demo'
        },
        {
            'url': 'https://example.com',
            'expected_type': None,
            'description': 'No CAPTCHA Expected'
        }
    ]
    
    # Initialize detection tool (simulated)
    if agentcore_browser:
        captcha_detector = CaptchaDetectionTool(agentcore_browser)
        
        for test_case in test_urls:
            print(f"\n🌐 Testing: {test_case['description']}")
            print(f"   URL: {test_case['url']}")
            print(f"   Expected: {test_case['expected_type'] or 'No CAPTCHA'}")
            
            try:
                # Simulate detection (in real scenario, this would call the actual tool)
                print("   🔍 Starting detection...")
                print("   🔒 Creating isolated AgentCore session...")
                print("   🌐 Navigating to target URL...")
                print("   📊 Analyzing DOM elements...")
                print("   📸 Capturing screenshots...")
                
                # Simulate results based on expected type
                if test_case['expected_type']:
                    print(f"   ✅ CAPTCHA Detected: {test_case['expected_type']}")
                    print(f"   🎯 Confidence Score: 0.95")
                    print(f"   📸 Screenshots captured (3 types)")
                    print(f"   🔒 Session recorded for replay")
                else:
                    print("   ℹ️ No CAPTCHA detected")
                    print("   ✅ Page ready for automation")
                
                print("   🧹 Cleaning up session...")
                
            except Exception as e:
                print(f"   ❌ Detection failed: {e}")
    
    else:
        print("⚠️ AgentCore Browser Tool not available - running simulation")
        
        for test_case in test_urls:
            print(f"\n🌐 Simulating: {test_case['description']}")
            
            if test_case['expected_type']:
                print(f"   ✅ Would detect: {test_case['expected_type']} CAPTCHA")
                print(f"   📊 Simulated confidence: 0.95")
            else:
                print("   ℹ️ Would detect: No CAPTCHA")
    
    print("\n🎉 Basic CAPTCHA Detection Demo Complete!")
    print("\n📋 Key Features Demonstrated:")
    print("   ✅ Multi-type CAPTCHA detection")
    print("   ✅ DOM element analysis")
    print("   ✅ Screenshot capture")
    print("   ✅ Confidence scoring")
    print("   ✅ AgentCore session management")
    print("   ✅ Error handling and cleanup")

# Run the demo
await demo_basic_captcha_detection()

## 7. Demo: AgentCore CAPTCHA Detection

Demonstrate CAPTCHA detection using AgentCore Browser Tool's enterprise features.

In [ ]:
# Demo AgentCore CAPTCHA detection capabilities
def demo_agentcore_captcha_features():
    """Demonstrate AgentCore Browser Tool features for CAPTCHA handling"""
    
    print("🎯 AgentCore Browser Tool CAPTCHA Handling Demo")
    print("=" * 60)
    
    # Simulate AgentCore session creation
    print("\n🔧 Creating Secure Browser Session:")
    print("  ✅ VM-level isolation: ENABLED")
    print("  ✅ Session recording: ENABLED")
    print("  ✅ CloudTrail logging: ENABLED")
    print("  ✅ Auto-scaling: ENABLED")
    print("  📊 Session ID: agentcore-session-abc123")
    
    # Simulate CAPTCHA detection
    test_scenarios = [
        {
            'url': 'https://www.google.com/recaptcha/api2/demo',
            'captcha_type': 'reCAPTCHA v2',
            'detected': True,
            'confidence': 0.95
        },
        {
            'url': 'https://accounts.hcaptcha.com/demo',
            'captcha_type': 'hCaptcha',
            'detected': True,
            'confidence': 0.92
        },
        {
            'url': 'https://example.com/contact-form',
            'captcha_type': None,
            'detected': False,
            'confidence': 0.0
        }
    ]
    
    for scenario in test_scenarios:
        print(f"\n🌐 Testing: {scenario['url']}")
        print(f"  🔍 Navigating in isolated session...")
        print(f"  📸 Taking screenshot for analysis...")
        
        if scenario['detected']:
            print(f"  ✅ CAPTCHA Detected: {scenario['captcha_type']}")
            print(f"  🎯 Confidence Score: {scenario['confidence']:.2f}")
            print(f"  🎬 Session replay available for debugging")
            print(f"  📋 CloudTrail logs recorded for compliance")
        else:
            print(f"  ℹ️ No CAPTCHA detected")
            print(f"  ✅ Page ready for automation")
    
    # Demonstrate observability features
    print("\n📊 AgentCore Observability Features:")
    print("  🎬 Session Replay: View complete browser session recording")
    print("  📋 CloudTrail Logs: Complete audit trail for compliance")
    print("  📈 Real-time Metrics: CPU, memory, and session statistics")
    print("  🔍 Live Monitoring: Watch sessions in real-time")
    print("  🚨 Alerting: Automated alerts for failures or anomalies")
    
    # Show cost benefits
    print("\n💰 Cost Benefits:")
    print("  ✅ No infrastructure to manage or maintain")
    print("  ✅ Pay only for active browser session time")
    print("  ✅ Automatic scaling reduces over-provisioning")
    print("  ✅ No idle resource costs")
    
    print("\n🔒 Security Advantages:")
    print("  ✅ Each session runs in isolated VM")
    print("  ✅ No data persistence between sessions")
    print("  ✅ Enterprise-grade network isolation")
    print("  ✅ Automatic security patching by AWS")
    
    print("\n🚀 Ready for Production CAPTCHA Handling!")

# Run the demo
demo_agentcore_captcha_features()

## 8. Strands Agent Integration

Integrate CAPTCHA detection with Strands agents and demonstrate different CAPTCHA types with ToolResult responses.

In [ ]:
class CaptchaHandlingAgent(Agent):
    """Strands agent with integrated CAPTCHA handling capabilities"""
    
    def __init__(self, config: AgentConfig):
        super().__init__(config)
        
        # Register CAPTCHA tools
        self.tool_registry = ToolRegistry()
        self.captcha_detector = None
        self.agentcore_client = None
        
        # Configure agent with CAPTCHA handling instructions
        self.system_prompt = self._build_captcha_aware_prompt()
    
    def setup_captcha_tools(self, agentcore_client):
        """Setup CAPTCHA detection tools with AgentCore client"""
        self.agentcore_client = agentcore_client
        self.captcha_detector = CaptchaDetectionTool(agentcore_client)
        
        # Register tools with the agent
        self.tool_registry.register(self.captcha_detector)
        
        print("✅ CAPTCHA tools registered with Strands agent")
    
    def _build_captcha_aware_prompt(self) -> str:
        """Build system prompt that includes CAPTCHA handling instructions"""
        return """
        You are a web automation agent with CAPTCHA handling capabilities using Amazon Bedrock AgentCore Browser Tool.
        
        When navigating websites, if you encounter CAPTCHAs:
        1. Use the captcha_detector tool to identify the CAPTCHA type and location
        2. Capture screenshots for analysis using AgentCore's secure browser sessions
        3. Provide detailed feedback about the CAPTCHA detection process
        4. If a CAPTCHA cannot be detected, explain the situation and suggest alternatives
        
        Always handle CAPTCHAs gracefully and provide clear feedback about the process.
        Leverage AgentCore's enterprise features like session isolation and observability.
        """
    
    async def navigate_with_captcha_detection(self, url: str, task: str) -> Dict[str, Any]:
        """Navigate to URL and detect CAPTCHAs with detailed Strands ToolResult responses"""
        
        print(f"🤖 Strands Agent starting navigation to: {url}")
        print(f"📋 Task: {task}")
        
        try:
            # Use CAPTCHA detection tool
            if self.captcha_detector:
                detection_result = await self.captcha_detector.execute(url)
                
                # Process ToolResult
                response = await self._process_captcha_detection_result(detection_result, task)
                return response
            else:
                return {
                    'success': False,
                    'error': 'CAPTCHA detection tools not initialized',
                    'message': 'Please setup CAPTCHA tools first'
                }
                
        except Exception as e:
            print(f"❌ Agent navigation failed: {e}")
            return {
                'success': False,
                'error': str(e),
                'message': f"Navigation failed: {e}"
            }
    
    async def _process_captcha_detection_result(self, tool_result: ToolResult, task: str) -> Dict[str, Any]:
        """Process CAPTCHA detection ToolResult and provide agent response"""
        
        if tool_result.success:
            captcha_data = tool_result.data
            
            if captcha_data['captcha_found']:
                return {
                    'success': True,
                    'captcha_detected': True,
                    'captcha_type': captcha_data['captcha_type'],
                    'confidence': captcha_data['confidence_score'],
                    'session_id': captcha_data['session_id'],
                    'message': f"CAPTCHA detected: {captcha_data['captcha_type']} (confidence: {captcha_data['confidence_score']:.2f})",
                    'next_steps': [
                        "CAPTCHA solving would be implemented in next phase",
                        "Screenshot captured for analysis",
                        "Session recorded for debugging"
                    ],
                    'agentcore_features': {
                        'session_isolation': 'enabled',
                        'observability': 'enabled',
                        'session_replay': 'available'
                    }
                }
            else:
                return {
                    'success': True,
                    'captcha_detected': False,
                    'message': 'No CAPTCHA detected - page ready for automation',
                    'next_steps': [f"Proceed with task: {task}"],
                    'session_id': captcha_data['session_id']
                }
        else:
            return {
                'success': False,
                'error': tool_result.error,
                'message': tool_result.message,
                'retry_suggested': True
            }

print("✅ CaptchaHandlingAgent implemented with Strands integration!")

## 9. CAPTCHA Type Examples with ToolResult Responses

Demonstrate different CAPTCHA types and their corresponding Strands ToolResult responses.

In [ ]:
def demonstrate_captcha_toolresults():
    """Demonstrate different CAPTCHA types with Strands ToolResult responses"""
    
    print("🎯 CAPTCHA TYPE EXAMPLES WITH TOOLRESULT RESPONSES")
    print("=" * 60)
    
    # Example ToolResult responses for different CAPTCHA types
    captcha_examples = [
        {
            'type': 'reCAPTCHA v2',
            'url': 'https://www.google.com/recaptcha/api2/demo',
            'toolresult': ToolResult(
                success=True,
                data={
                    'captcha_found': True,
                    'captcha_type': 'recaptcha_v2',
                    'element_selector': '.g-recaptcha',
                    'confidence_score': 0.95,
                    'screenshot_data': 'base64_encoded_screenshot_data...',
                    'page_url': 'https://www.google.com/recaptcha/api2/demo',
                    'session_id': 'agentcore-session-rec123',
                    'detection_timestamp': '2024-01-15T10:30:00Z',
                    'additional_info': {
                        'sitekey': '6Le-wvkSAAAAAPBMRTvw0Q4Muexq9bi0DJwx_mJ-',
                        'theme': 'light',
                        'size': 'normal'
                    }
                },
                message="reCAPTCHA v2 detected with high confidence"
            )
        },
        {
            'type': 'hCaptcha',
            'url': 'https://accounts.hcaptcha.com/demo',
            'toolresult': ToolResult(
                success=True,
                data={
                    'captcha_found': True,
                    'captcha_type': 'hcaptcha',
                    'element_selector': '.h-captcha',
                    'confidence_score': 0.92,
                    'screenshot_data': 'base64_encoded_screenshot_data...',
                    'page_url': 'https://accounts.hcaptcha.com/demo',
                    'session_id': 'agentcore-session-hcap456',
                    'detection_timestamp': '2024-01-15T10:35:00Z',
                    'additional_info': {
                        'sitekey': '10000000-ffff-ffff-ffff-000000000001',
                        'theme': 'light',
                        'size': 'normal'
                    }
                },
                message="hCaptcha detected with high confidence"
            )
        },
        {
            'type': 'Image CAPTCHA',
            'url': 'https://example.com/contact-form',
            'toolresult': ToolResult(
                success=True,
                data={
                    'captcha_found': True,
                    'captcha_type': 'image_captcha',
                    'element_selector': 'img[alt="captcha"]',
                    'confidence_score': 0.88,
                    'screenshot_data': 'base64_encoded_screenshot_data...',
                    'page_url': 'https://example.com/contact-form',
                    'session_id': 'agentcore-session-img789',
                    'detection_timestamp': '2024-01-15T10:40:00Z',
                    'additional_info': {
                        'image_dimensions': {'width': 200, 'height': 80},
                        'input_field': 'input[name="captcha_code"]',
                        'refresh_button': 'button[onclick="refreshCaptcha()"]'
                    }
                },
                message="Image-based CAPTCHA detected"
            )
        },
        {
            'type': 'No CAPTCHA',
            'url': 'https://example.com/simple-form',
            'toolresult': ToolResult(
                success=True,
                data={
                    'captcha_found': False,
                    'captcha_type': None,
                    'element_selector': None,
                    'confidence_score': 0.0,
                    'screenshot_data': 'base64_encoded_screenshot_data...',
                    'page_url': 'https://example.com/simple-form',
                    'session_id': 'agentcore-session-none101',
                    'detection_timestamp': '2024-01-15T10:45:00Z'
                },
                message="No CAPTCHA detected - page ready for automation"
            )
        }
    ]
    
    # Display each example
    for i, example in enumerate(captcha_examples, 1):
        print(f"\n{i}. {example['type']} Example")
        print(f"   URL: {example['url']}")
        print(f"   ToolResult Success: {example['toolresult'].success}")
        print(f"   Message: {example['toolresult'].message}")
        
        if example['toolresult'].data:
            data = example['toolresult'].data
            print(f"   CAPTCHA Found: {data['captcha_found']}")
            
            if data['captcha_found']:
                print(f"   CAPTCHA Type: {data['captcha_type']}")
                print(f"   Confidence: {data['confidence_score']:.2f}")
                print(f"   Selector: {data['element_selector']}")
                print(f"   Session ID: {data['session_id']}")
                
                if 'additional_info' in data:
                    print(f"   Additional Info: {data['additional_info']}")
            else:
                print("   ✅ Page ready for automation")
        
        print("   " + "-" * 40)
    
    print("\n📋 ToolResult Response Structure:")
    print("   ✅ success: Boolean indicating detection success")
    print("   📊 data: Dictionary with detection details")
    print("   💬 message: Human-readable status message")
    print("   ❌ error: Error details (if success=False)")
    
    print("\n🔍 Data Fields Explained:")
    print("   • captcha_found: Boolean - CAPTCHA presence")
    print("   • captcha_type: String - Type of CAPTCHA detected")
    print("   • element_selector: String - CSS selector for CAPTCHA element")
    print("   • confidence_score: Float - Detection confidence (0.0-1.0)")
    print("   • screenshot_data: String - Base64 encoded screenshot")
    print("   • session_id: String - AgentCore session identifier")
    print("   • detection_timestamp: String - ISO timestamp")
    print("   • additional_info: Dict - Type-specific metadata")

# Run the demonstration
demonstrate_captcha_toolresults()

## 10. Error Handling Examples

Demonstrate error handling for CAPTCHA detection failures within the Strands framework.

In [ ]:
class StrandsCaptchaErrorHandler:
    """Error handling for CAPTCHA detection within Strands framework"""
    
    @staticmethod
    def demonstrate_error_scenarios():
        """Demonstrate various error scenarios and their ToolResult responses"""
        
        print("🚨 CAPTCHA DETECTION ERROR HANDLING EXAMPLES")
        print("=" * 55)
        
        error_scenarios = [
            {
                'scenario': 'Network Timeout',
                'description': 'AgentCore session times out during navigation',
                'toolresult': ToolResult(
                    success=False,
                    error='SessionTimeoutError: AgentCore session timed out after 300 seconds',
                    message='CAPTCHA detection failed due to session timeout',
                    data={
                        'error_type': 'timeout',
                        'session_id': 'agentcore-session-timeout123',
                        'retry_suggested': True,
                        'retry_delay': 30
                    }
                )
            },
            {
                'scenario': 'Invalid URL',
                'description': 'Target URL is malformed or unreachable',
                'toolresult': ToolResult(
                    success=False,
                    error='NavigationError: Unable to navigate to invalid URL',
                    message='CAPTCHA detection failed: Invalid or unreachable URL',
                    data={
                        'error_type': 'navigation',
                        'url': 'https://invalid-domain-12345.com',
                        'retry_suggested': False,
                        'suggestion': 'Verify URL format and accessibility'
                    }
                )
            },
            {
                'scenario': 'DOM Analysis Failure',
                'description': 'Error during DOM element detection',
                'toolresult': ToolResult(
                    success=False,
                    error='DOMAnalysisError: Unable to analyze page DOM structure',
                    message='CAPTCHA detection failed during DOM analysis',
                    data={
                        'error_type': 'dom_analysis',
                        'session_id': 'agentcore-session-dom456',
                        'page_loaded': True,
                        'retry_suggested': True,
                        'fallback_options': ['screenshot_analysis', 'manual_inspection']
                    }
                )
            },
            {
                'scenario': 'Screenshot Capture Failure',
                'description': 'Unable to capture screenshots for analysis',
                'toolresult': ToolResult(
                    success=True,  # Detection succeeded, but screenshot failed
                    data={
                        'captcha_found': True,
                        'captcha_type': 'recaptcha_v2',
                        'confidence_score': 0.90,
                        'screenshot_data': None,
                        'screenshot_error': 'ScreenshotError: Unable to capture element screenshot',
                        'session_id': 'agentcore-session-screen789'
                    },
                    message='CAPTCHA detected but screenshot capture failed'
                )
            },
            {
                'scenario': 'AgentCore Service Unavailable',
                'description': 'AgentCore Browser Tool service is unavailable',
                'toolresult': ToolResult(
                    success=False,
                    error='ServiceUnavailableError: AgentCore Browser Tool service is temporarily unavailable',
                    message='CAPTCHA detection failed: AgentCore service unavailable',
                    data={
                        'error_type': 'service_unavailable',
                        'service_status': 'unavailable',
                        'retry_suggested': True,
                        'retry_delay': 300,  # 5 minutes
                        'alternative_options': ['local_browser_fallback', 'queue_for_later']
                    }
                )
            },
            {
                'scenario': 'Insufficient Permissions',
                'description': 'AWS credentials lack AgentCore permissions',
                'toolresult': ToolResult(
                    success=False,
                    error='PermissionError: Insufficient permissions for AgentCore Browser Tool',
                    message='CAPTCHA detection failed: Permission denied',
                    data={
                        'error_type': 'permission_denied',
                        'required_permissions': [
                            'bedrock:InvokeAgent',
                            'agentcore:CreateSession',
                            'agentcore:NavigatePage'
                        ],
                        'retry_suggested': False,
                        'resolution': 'Update IAM permissions for AgentCore access'
                    }
                )
            }
        ]
        
        # Display each error scenario
        for i, scenario in enumerate(error_scenarios, 1):
            print(f"\n{i}. {scenario['scenario']}")
            print(f"   Description: {scenario['description']}")
            
            result = scenario['toolresult']
            print(f"   Success: {result.success}")
            print(f"   Message: {result.message}")
            
            if result.error:
                print(f"   Error: {result.error}")
            
            if result.data:
                data = result.data
                if 'error_type' in data:
                    print(f"   Error Type: {data['error_type']}")
                if 'retry_suggested' in data:
                    print(f"   Retry Suggested: {data['retry_suggested']}")
                if 'retry_delay' in data:
                    print(f"   Retry Delay: {data['retry_delay']} seconds")
                if 'suggestion' in data:
                    print(f"   Suggestion: {data['suggestion']}")
                if 'alternative_options' in data:
                    print(f"   Alternatives: {', '.join(data['alternative_options'])}")
            
            print("   " + "-" * 50)
        
        return error_scenarios
    
    @staticmethod
    def demonstrate_error_recovery_patterns():
        """Demonstrate error recovery patterns for Strands agents"""
        
        print("\n🔄 ERROR RECOVERY PATTERNS")
        print("=" * 35)
        
        recovery_patterns = [
            {
                'pattern': 'Exponential Backoff Retry',
                'use_case': 'Temporary service unavailability',
                'implementation': '''
async def retry_with_backoff(func, max_retries=3):
    for attempt in range(max_retries):
        try:
            result = await func()
            if result.success:
                return result
        except Exception as e:
            if attempt == max_retries - 1:
                raise e
            
            delay = 2 ** attempt  # Exponential backoff
            await asyncio.sleep(delay)
    
    return ToolResult(success=False, error="Max retries exceeded")
                '''
            },
            {
                'pattern': 'Graceful Degradation',
                'use_case': 'Screenshot capture failure',
                'implementation': '''
async def detect_with_fallback(url):
    try:
        # Primary detection with screenshots
        result = await full_captcha_detection(url)
        return result
    except ScreenshotError:
        # Fallback to DOM-only detection
        result = await dom_only_detection(url)
        result.data['screenshot_warning'] = "Screenshots unavailable"
        return result
                '''
            },
            {
                'pattern': 'Circuit Breaker',
                'use_case': 'Persistent service failures',
                'implementation': '''
class CaptchaDetectionCircuitBreaker:
    def __init__(self, failure_threshold=5, timeout=300):
        self.failure_count = 0
        self.failure_threshold = failure_threshold
        self.timeout = timeout
        self.last_failure_time = None
        self.state = "CLOSED"  # CLOSED, OPEN, HALF_OPEN
    
    async def call(self, func):
        if self.state == "OPEN":
            if time.time() - self.last_failure_time > self.timeout:
                self.state = "HALF_OPEN"
            else:
                return ToolResult(success=False, error="Circuit breaker OPEN")
        
        try:
            result = await func()
            if result.success:
                self.failure_count = 0
                self.state = "CLOSED"
            return result
        except Exception as e:
            self.failure_count += 1
            self.last_failure_time = time.time()
            
            if self.failure_count >= self.failure_threshold:
                self.state = "OPEN"
            
            raise e
                '''
            }
        ]
        
        for pattern in recovery_patterns:
            print(f"\n📋 {pattern['pattern']}")
            print(f"   Use Case: {pattern['use_case']}")
            print(f"   Implementation:")
            print(pattern['implementation'])
        
        print("\n✅ Error Recovery Best Practices:")
        print("   • Always provide meaningful error messages")
        print("   • Include retry suggestions when appropriate")
        print("   • Implement graceful degradation for non-critical failures")
        print("   • Use circuit breakers for persistent service issues")
        print("   • Log errors for debugging and monitoring")
        print("   • Provide alternative workflows when possible")

# Demonstrate error handling
error_handler = StrandsCaptchaErrorHandler()
error_scenarios = error_handler.demonstrate_error_scenarios()
error_handler.demonstrate_error_recovery_patterns()

print("\n🎉 Error Handling Examples Complete!")

## 11. Integration Demo

Demonstrate complete Strands agent integration with CAPTCHA detection.

In [ ]:
async def demo_strands_captcha_integration():
    """Demonstrate complete Strands agent integration with CAPTCHA detection"""
    
    print("🤖 STRANDS AGENT CAPTCHA INTEGRATION DEMO")
    print("=" * 50)
    
    # Create agent configuration
    agent_config = AgentConfig(
        name="CaptchaHandlingAgent",
        description="Strands agent with CAPTCHA detection capabilities",
        model_config={
            'provider': 'bedrock',
            'model_name': 'anthropic.claude-3-sonnet-20240229-v1:0',
            'region': 'us-east-1'
        },
        tools_config={
            'captcha_detection': True,
            'agentcore_browser': True
        }
    )
    
    # Initialize agent
    print("\n🔧 Initializing Strands Agent...")
    agent = CaptchaHandlingAgent(agent_config)
    
    # Setup CAPTCHA tools (simulated)
    if agentcore_browser:
        agent.setup_captcha_tools(agentcore_browser)
        print("✅ CAPTCHA tools configured with AgentCore Browser Tool")
    else:
        print("⚠️ AgentCore Browser Tool not available - using simulation")
    
    # Test scenarios
    test_scenarios = [
        {
            'url': 'https://www.google.com/recaptcha/api2/demo',
            'task': 'Fill out the demo form',
            'expected_captcha': 'recaptcha_v2'
        },
        {
            'url': 'https://accounts.hcaptcha.com/demo',
            'task': 'Complete the hCaptcha demo',
            'expected_captcha': 'hcaptcha'
        },
        {
            'url': 'https://example.com/simple-form',
            'task': 'Submit contact information',
            'expected_captcha': None
        }
    ]
    
    # Execute test scenarios
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\n🌐 Scenario {i}: {scenario['task']}")
        print(f"   URL: {scenario['url']}")
        print(f"   Expected CAPTCHA: {scenario['expected_captcha'] or 'None'}")
        
        try:
            # Simulate agent navigation with CAPTCHA detection
            print("   🤖 Agent starting navigation...")
            
            if agentcore_browser:
                # Real execution would happen here
                result = await agent.navigate_with_captcha_detection(
                    scenario['url'], 
                    scenario['task']
                )
            else:
                # Simulate results
                if scenario['expected_captcha']:
                    result = {
                        'success': True,
                        'captcha_detected': True,
                        'captcha_type': scenario['expected_captcha'],
                        'confidence': 0.95,
                        'session_id': f'sim-session-{i}',
                        'message': f"CAPTCHA detected: {scenario['expected_captcha']}",
                        'agentcore_features': {
                            'session_isolation': 'enabled',
                            'observability': 'enabled',
                            'session_replay': 'available'
                        }
                    }
                else:
                    result = {
                        'success': True,
                        'captcha_detected': False,
                        'message': 'No CAPTCHA detected - page ready for automation',
                        'session_id': f'sim-session-{i}'
                    }
            
            # Display results
            print(f"   ✅ Navigation Success: {result['success']}")
            print(f"   🔍 CAPTCHA Detected: {result.get('captcha_detected', False)}")
            
            if result.get('captcha_detected'):
                print(f"   🎯 CAPTCHA Type: {result['captcha_type']}")
                print(f"   📊 Confidence: {result['confidence']:.2f}")
                print(f"   🔒 Session ID: {result['session_id']}")
                
                if 'agentcore_features' in result:
                    features = result['agentcore_features']
                    print(f"   🛡️ Isolation: {features['session_isolation']}")
                    print(f"   📊 Observability: {features['observability']}")
                    print(f"   🎬 Session Replay: {features['session_replay']}")
            else:
                print(f"   ✅ Ready for automation")
            
            print(f"   💬 Message: {result['message']}")
            
        except Exception as e:
            print(f"   ❌ Scenario failed: {e}")
        
        print("   " + "-" * 40)
    
    print("\n🎉 Strands Agent Integration Demo Complete!")
    
    print("\n📋 Integration Features Demonstrated:")
    print("   ✅ Strands Agent with CAPTCHA tools")
    print("   ✅ AgentCore Browser Tool integration")
    print("   ✅ ToolResult response handling")
    print("   ✅ Error handling and recovery")
    print("   ✅ Enterprise security features")
    print("   ✅ Session isolation and observability")
    
    print("\n🚀 Next Steps:")
    print("   1. Implement CAPTCHA solving capabilities")
    print("   2. Add workflow orchestration")
    print("   3. Integrate with Bedrock vision models")
    print("   4. Deploy to production environment")

# Run the integration demo
await demo_strands_captcha_integration()

## 12. Tutorial Summary

Key takeaways from using Strands with AgentCore Browser Tool for CAPTCHA handling.

In [ ]:
def display_tutorial_summary():
    """Display comprehensive tutorial summary"""
    
    print("🎯 STRANDS + AGENTCORE BROWSER TOOL TUTORIAL COMPLETE")
    print("=" * 70)
    
    print("\n✅ WHAT YOU'VE LEARNED:")
    print("\n🔒 Enterprise Security:")
    print("  • VM-level isolation for each CAPTCHA session")
    print("  • No data leakage between browser sessions")
    print("  • Enterprise-grade security compliance")
    print("  • Automatic security patching by AWS")
    
    print("\n📈 Serverless Scaling:")
    print("  • Automatic scaling from 1 to thousands of sessions")
    print("  • No infrastructure management required")
    print("  • Consumption-based pricing model")
    print("  • Zero idle resource costs")
    
    print("\n📊 Built-in Observability:")
    print("  • Session replay for debugging CAPTCHA failures")
    print("  • CloudTrail logging for compliance and audit")
    print("  • Real-time monitoring and metrics")
    print("  • Live session viewing capabilities")
    
    print("\n🤖 Strands Integration:")
    print("  • Native AgentCore Browser Tool support")
    print("  • Seamless tool and workflow integration")
    print("  • Production-ready error handling patterns")
    print("  • Enterprise deployment capabilities")
    
    print("\n💡 KEY ADVANTAGES vs Traditional Browser Automation:")
    print("  ✅ No browser infrastructure to manage")
    print("  ✅ Enterprise-grade isolation and security")
    print("  ✅ Built-in observability and compliance features")
    print("  ✅ Automatic scaling without configuration")
    print("  ✅ Session replay for debugging")
    print("  ✅ Consumption-based cost model")
    print("  ✅ AWS-managed security and updates")
    
    print("\n🚀 NEXT STEPS:")
    print("\n1. Production Deployment:")
    print("   • Set up AWS credentials and AgentCore permissions")
    print("   • Configure session pools for your workload")
    print("   • Implement monitoring and alerting")
    
    print("2. Advanced Features:")
    print("   • Integrate with Bedrock vision models for CAPTCHA solving")
    print("   • Implement custom CAPTCHA detection algorithms")
    print("   • Add multi-region deployment for global scale")
    
    print("3. Compliance and Security:")
    print("   • Review CloudTrail logs for audit requirements")
    print("   • Implement data retention policies")
    print("   • Set up compliance monitoring dashboards")
    
    print("\n🎉 Congratulations!")
    print("You're ready to build enterprise-grade CAPTCHA automation with:")
    print("   🔒 VM-level isolation")
    print("   📈 Serverless auto-scaling")
    print("   📊 Built-in observability")
    print("   🤖 Strands framework integration")
    print("\n🚀 Start building your production CAPTCHA handling solution today!")

# Display the summary
display_tutorial_summary()

## 4. AI-Powered CAPTCHA Solving with Bedrock Vision Models

Now let's implement the complete AI-powered CAPTCHA solving workflow using Bedrock Vision models integrated with AgentCore Browser Tool screenshots.

In [ ]:
# Import the AI-powered CAPTCHA solving modules
from strands_captcha_vision import BedrockVisionClient, CaptchaImageProcessor
from strands_captcha_solver import CaptchaSolvingTool, CaptchaSubmissionTool
from strands_captcha_integration import EndToEndCaptchaSolver, CaptchaWorkflowOrchestrator

print("✅ AI-powered CAPTCHA solving modules imported!")
print("🧠 Ready for Bedrock Vision model integration!")

### 4.1 Bedrock Vision Model Integration

Set up Bedrock Vision models for analyzing CAPTCHA screenshots captured by AgentCore Browser Tool.

In [ ]:
# Initialize Bedrock Vision client for CAPTCHA analysis
def setup_bedrock_vision_integration():
    """Set up Bedrock Vision models for CAPTCHA analysis"""
    
    try:
        print("🧠 Initializing Bedrock Vision integration...")
        
        # Initialize vision client
        vision_client = BedrockVisionClient(region_name='us-east-1')
        
        # Test vision model availability
        available_models = vision_client.vision_models
        print(f"📋 Available vision models: {list(available_models.keys())}")
        
        # Initialize image processor
        image_processor = CaptchaImageProcessor()
        
        print("✅ Bedrock Vision integration ready!")
        print("🎯 Supported CAPTCHA types:")
        print("   • Text-based CAPTCHAs (OCR analysis)")
        print("   • Image selection CAPTCHAs (object detection)")
        print("   • reCAPTCHA challenges (multi-modal analysis)")
        print("   • hCaptcha challenges (pattern recognition)")
        print("   • Mathematical CAPTCHAs (equation solving)")
        
        return vision_client, image_processor
        
    except Exception as e:
        print(f"❌ Bedrock Vision setup failed: {e}")
        print("💡 Ensure AWS credentials and Bedrock permissions are configured")
        return None, None

# Set up Bedrock Vision integration
vision_client, image_processor = setup_bedrock_vision_integration()

### 4.2 End-to-End CAPTCHA Solving Workflow

Implement the complete workflow that combines AgentCore screenshots with Bedrock AI analysis.

In [ ]:
# Create end-to-end CAPTCHA solver
class CompleteCaptchaSolution:
    """Complete CAPTCHA solving solution using Strands + AgentCore + Bedrock"""
    
    def __init__(self, agentcore_client):
        self.agentcore = agentcore_client
        
        # Initialize AI-powered solver
        self.end_to_end_solver = EndToEndCaptchaSolver(agentcore_client)
        
        # Initialize workflow orchestrator
        self.workflow_orchestrator = CaptchaWorkflowOrchestrator(agentcore_client)
        
        print("✅ Complete CAPTCHA solution initialized!")
    
    async def solve_text_captcha_demo(self):
        """Demonstrate text-based CAPTCHA solving"""
        
        print("🔤 Demonstrating Text CAPTCHA Solving...")
        print("=" * 50)
        
        # Mock CAPTCHA data for demonstration
        text_captcha_data = {
            'captcha_type': 'text_captcha',
            'element_selector': 'input[name="captcha"]',
            'bounds': {'x': 100, 'y': 200, 'width': 200, 'height': 60},
            'difficulty': 'medium'
        }
        
        try:
            # Execute end-to-end solving
            result = await self.end_to_end_solver.execute(
                captcha_data=text_captcha_data,
                auto_submit=True,
                confidence_threshold=0.7,
                model_config={
                    'model_name': 'claude-3-sonnet',
                    'focus': 'text_recognition',
                    'temperature': 0.1
                }
            )
            
            if result.success:
                workflow_data = result.data
                solution_data = workflow_data['solution_data']
                metrics = workflow_data['workflow_metrics']
                
                print(f"✅ Text CAPTCHA solved successfully!")
                print(f"   Solution: '{solution_data['solution']}'")
                print(f"   Confidence: {solution_data['confidence']:.2f}")
                print(f"   Model used: {solution_data['model_used']}")
                print(f"   Processing time: {metrics['total_time']:.2f}s")
                print(f"   Auto-submitted: {metrics['auto_submitted']}")
            else:
                print(f"❌ Text CAPTCHA solving failed: {result.error}")
                
        except Exception as e:
            print(f"❌ Text CAPTCHA demo failed: {e}")
    
    async def solve_image_selection_demo(self):
        """Demonstrate image selection CAPTCHA solving"""
        
        print("\n🖼️ Demonstrating Image Selection CAPTCHA Solving...")
        print("=" * 50)
        
        # Mock image selection CAPTCHA data
        image_captcha_data = {
            'captcha_type': 'image_selection',
            'element_selector': '.captcha-images',
            'container_selector': '.image-grid',
            'bounds': {'x': 50, 'y': 150, 'width': 300, 'height': 300},
            'instruction': 'Select all images with traffic lights',
            'grid_size': '3x3'
        }
        
        try:
            # Execute with image-optimized configuration
            result = await self.end_to_end_solver.execute(
                captcha_data=image_captcha_data,
                auto_submit=True,
                confidence_threshold=0.6,  # Lower threshold for image tasks
                model_config={
                    'model_name': 'claude-3-opus',  # More powerful model for images
                    'focus': 'object_detection',
                    'temperature': 0.2
                }
            )
            
            if result.success:
                workflow_data = result.data
                solution_data = workflow_data['solution_data']
                metrics = workflow_data['workflow_metrics']
                
                print(f"✅ Image selection CAPTCHA solved successfully!")
                print(f"   Selected tiles: {solution_data['solution']}")
                print(f"   Confidence: {solution_data['confidence']:.2f}")
                print(f"   Model used: {solution_data['model_used']}")
                print(f"   Processing time: {metrics['total_time']:.2f}s")
                
                # Show tile analysis
                if 'solution_metadata' in solution_data:
                    metadata = solution_data['solution_metadata']
                    if 'selected_tiles' in metadata:
                        print(f"   Tiles selected: {metadata['selected_tiles']} out of 9")
            else:
                print(f"❌ Image selection CAPTCHA solving failed: {result.error}")
                
        except Exception as e:
            print(f"❌ Image selection demo failed: {e}")
    
    async def demonstrate_workflow_orchestration(self):
        """Demonstrate complete workflow orchestration"""
        
        print("\n🎯 Demonstrating Complete Workflow Orchestration...")
        print("=" * 50)
        
        # Complex CAPTCHA scenario
        complex_captcha_data = {
            'captcha_type': 'recaptcha',
            'element_selector': '.g-recaptcha',
            'bounds': {'x': 75, 'y': 175, 'width': 300, 'height': 200},
            'challenge_type': 'image_selection',
            'instruction': 'Select all squares with crosswalks'
        }
        
        # Workflow configuration
        workflow_config = {
            'auto_submit': True,
            'confidence_threshold': 0.75,
            'max_retries': 2,
            'model_preference': 'claude-3-opus'
        }
        
        try:
            from strands.workflows import WorkflowContext
            
            # Create workflow context
            context = WorkflowContext({
                'captcha_data': complex_captcha_data,
                'config': workflow_config,
                'start_time': time.time()
            })
            
            print("🚀 Executing complete workflow orchestration...")
            
            # Execute workflow
            result_context = await self.workflow_orchestrator.execute(context)
            
            # Analyze results
            final_result = result_context.get('final_result', {})
            
            print(f"\n📊 Workflow Orchestration Results:")
            print(f"   Overall Success: {final_result.get('overall_success', False)}")
            
            phases = final_result.get('phases_completed', {})
            print(f"   Validation: {'✅' if phases.get('validation') else '❌'}")
            print(f"   Solving: {'✅' if phases.get('solving') else '❌'}")
            print(f"   Verification: {'✅' if phases.get('verification') else '❌'}")
            
            metrics = final_result.get('metrics', {})
            print(f"   Confidence Score: {metrics.get('confidence_score', 0):.2f}")
            print(f"   Processing Time: {metrics.get('processing_time', 0):.2f}s")
            print(f"   Total Workflow Time: {metrics.get('total_workflow_time', 0):.2f}s")
            
            # Show any errors
            errors = final_result.get('errors', {})
            for phase, error in errors.items():
                if error:
                    print(f"   {phase.title()} Error: {error}")
                    
        except Exception as e:
            print(f"❌ Workflow orchestration demo failed: {e}")

# Initialize complete solution (using mock client for demo)
if agentcore_browser:  # Use real client if available
    complete_solution = CompleteCaptchaSolution(agentcore_browser)
else:
    # Create mock client for demonstration
    class MockAgentCoreForDemo:
        async def capture_element_screenshot(self, selector, padding=0):
            from PIL import Image
            import io
            
            # Create realistic CAPTCHA mock image
            img = Image.new('RGB', (200, 60), color='lightblue')
            buffer = io.BytesIO()
            img.save(buffer, format='PNG')
            return buffer.getvalue()
        
        async def element_exists(self, selector):
            return True
        
        async def click_element(self, selector):
            print(f"Demo: Clicked {selector}")
        
        async def type_text(self, selector, text):
            print(f"Demo: Typed '{text}' into {selector}")
        
        async def clear_element(self, selector):
            print(f"Demo: Cleared {selector}")
    
    complete_solution = CompleteCaptchaSolution(MockAgentCoreForDemo())

print("✅ Complete CAPTCHA solution ready for demonstration!")

### 4.3 Run AI-Powered CAPTCHA Solving Demonstrations

Execute the complete AI-powered CAPTCHA solving workflows.

In [ ]:
# Run comprehensive CAPTCHA solving demonstrations
async def run_ai_captcha_demonstrations():
    """Run all AI-powered CAPTCHA solving demonstrations"""
    
    print("🧠 AI-POWERED CAPTCHA SOLVING DEMONSTRATIONS")
    print("=" * 60)
    
    try:
        # Demonstrate text CAPTCHA solving
        await complete_solution.solve_text_captcha_demo()
        
        # Demonstrate image selection CAPTCHA solving
        await complete_solution.solve_image_selection_demo()
        
        # Demonstrate complete workflow orchestration
        await complete_solution.demonstrate_workflow_orchestration()
        
        print("\n🎉 All AI-powered CAPTCHA demonstrations completed successfully!")
        
        # Show integration benefits
        print("\n💡 KEY INTEGRATION BENEFITS:")
        print("\n🔒 Security & Isolation:")
        print("   • AgentCore provides VM-level isolation for each CAPTCHA session")
        print("   • Bedrock ensures secure AI model access with enterprise controls")
        print("   • No CAPTCHA data stored locally or in browser cache")
        
        print("\n🧠 AI-Powered Analysis:")
        print("   • Claude Vision models provide state-of-the-art CAPTCHA analysis")
        print("   • Multi-modal understanding for complex CAPTCHA types")
        print("   • Confidence scoring for solution quality assessment")
        
        print("\n📈 Scalability & Performance:")
        print("   • Serverless architecture scales automatically")
        print("   • Parallel CAPTCHA processing capabilities")
        print("   • Optimized image preprocessing for faster analysis")
        
        print("\n🔧 Production Ready:")
        print("   • Comprehensive error handling and retry logic")
        print("   • Built-in observability and monitoring")
        print("   • Workflow orchestration for complex scenarios")
        
    except Exception as e:
        print(f"❌ AI CAPTCHA demonstrations failed: {e}")

# Run the demonstrations
await run_ai_captcha_demonstrations()

### 4.4 Confidence Scoring and Solution Validation

Understand how the AI system validates and scores CAPTCHA solutions.

In [ ]:
# Demonstrate confidence scoring and validation
def demonstrate_confidence_scoring():
    """Show how confidence scoring works in AI CAPTCHA solving"""
    
    print("📊 CONFIDENCE SCORING & SOLUTION VALIDATION")
    print("=" * 50)
    
    print("\n🎯 Confidence Score Factors:")
    print("\n1. Vision Model Confidence:")
    print("   • Base confidence from Claude Vision model analysis")
    print("   • Adjusted based on CAPTCHA type and complexity")
    print("   • Higher confidence for clear, unambiguous CAPTCHAs")
    
    print("\n2. Solution Format Validation:")
    print("   • Text CAPTCHAs: Length and character validation")
    print("   • Image Selection: Tile count and range validation")
    print("   • Mathematical: Numerical result validation")
    
    print("\n3. Pattern Recognition Adjustments:")
    print("   • Boost confidence for typical CAPTCHA patterns")
    print("   • Penalize unusual or suspicious solutions")
    print("   • Consider historical success rates")
    
    print("\n📈 Confidence Thresholds:")
    confidence_levels = {
        0.9: "Excellent - Very high confidence, proceed immediately",
        0.8: "Good - High confidence, safe to proceed", 
        0.7: "Acceptable - Standard threshold for most scenarios",
        0.6: "Moderate - Consider retry with different model",
        0.5: "Low - Retry recommended before submission",
        0.3: "Very Low - Manual review or alternative approach needed"
    }
    
    for threshold, description in confidence_levels.items():
        print(f"   {threshold:.1f}+: {description}")
    
    print("\n🔄 Adaptive Retry Strategy:")
    print("   • Low confidence triggers automatic retry with enhanced model")
    print("   • Different models used for different CAPTCHA types")
    print("   • Exponential backoff for failed attempts")
    print("   • Maximum retry limits to prevent infinite loops")
    
    print("\n✅ Solution Validation Process:")
    validation_steps = [
        "1. Parse AI model response and extract solution",
        "2. Validate solution format based on CAPTCHA type",
        "3. Apply confidence adjustments based on validation",
        "4. Check against confidence threshold",
        "5. Determine if solution is ready for submission",
        "6. Execute retry logic if confidence is insufficient"
    ]
    
    for step in validation_steps:
        print(f"   {step}")
    
    print("\n💡 Best Practices:")
    print("   • Use higher thresholds (0.8+) for critical applications")
    print("   • Lower thresholds (0.6+) acceptable for non-critical scenarios")
    print("   • Monitor success rates and adjust thresholds accordingly")
    print("   • Implement fallback strategies for consistently low confidence")

# Show confidence scoring explanation
demonstrate_confidence_scoring()

## 5. Production Deployment and Best Practices

Guidelines for deploying AI-powered CAPTCHA solving in production environments.

In [ ]:
# Production deployment guidelines
def show_production_guidelines():
    """Display production deployment guidelines for AI CAPTCHA solving"""
    
    print("🚀 PRODUCTION DEPLOYMENT GUIDELINES")
    print("=" * 50)
    
    print("\n🔧 Infrastructure Setup:")
    print("\n1. AWS Permissions:")
    print("   • bedrock:InvokeModel for vision model access")
    print("   • agentcore:CreateSession for browser sessions")
    print("   • agentcore:CaptureScreenshot for image capture")
    print("   • logs:CreateLogGroup for observability")
    
    print("\n2. Resource Configuration:")
    print("   • Configure session pools based on expected load")
    print("   • Set appropriate timeout values (5-10 minutes)")
    print("   • Enable auto-scaling for variable workloads")
    print("   • Configure multi-region deployment for global access")
    
    print("\n📊 Monitoring & Observability:")
    print("\n1. Key Metrics to Track:")
    metrics = [
        "CAPTCHA solving success rate",
        "Average confidence scores",
        "Processing time per CAPTCHA type",
        "Retry rates and failure reasons",
        "AgentCore session utilization",
        "Bedrock model invocation costs"
    ]
    
    for metric in metrics:
        print(f"   • {metric}")
    
    print("\n2. Alerting Setup:")
    print("   • Success rate drops below 85%")
    print("   • Average processing time exceeds 30 seconds")
    print("   • High retry rates (>20% of attempts)")
    print("   • AgentCore session pool exhaustion")
    
    print("\n🔒 Security Best Practices:")
    print("\n1. Data Protection:")
    print("   • Never log CAPTCHA images or solutions")
    print("   • Use encrypted storage for temporary data")
    print("   • Implement data retention policies")
    print("   • Regular security audits and compliance checks")
    
    print("\n2. Access Control:")
    print("   • Principle of least privilege for IAM roles")
    print("   • Network isolation using VPCs")
    print("   • API authentication and rate limiting")
    print("   • Regular credential rotation")
    
    print("\n⚖️ Ethical & Legal Considerations:")
    print("\n1. Compliance Requirements:")
    print("   • Respect website terms of service")
    print("   • Implement appropriate rate limiting")
    print("   • Avoid overwhelming target systems")
    print("   • Maintain audit trails for compliance")
    
    print("\n2. Responsible Usage:")
    print("   • Use only for legitimate automation purposes")
    print("   • Implement circuit breakers for failed attempts")
    print("   • Provide manual fallback options")
    print("   • Regular review of usage patterns")
    
    print("\n💰 Cost Optimization:")
    print("\n1. Bedrock Usage:")
    print("   • Choose appropriate model sizes for CAPTCHA complexity")
    print("   • Implement caching for repeated CAPTCHA patterns")
    print("   • Use batch processing where possible")
    print("   • Monitor and optimize token usage")
    
    print("\n2. AgentCore Sessions:")
    print("   • Optimize session lifecycle management")
    print("   • Implement session pooling and reuse")
    print("   • Configure appropriate timeout values")
    print("   • Use spot instances for non-critical workloads")
    
    print("\n🔧 Performance Optimization:")
    print("\n1. Image Processing:")
    print("   • Optimize image compression for faster uploads")
    print("   • Implement intelligent cropping to reduce data")
    print("   • Use appropriate image formats (PNG vs JPEG)")
    print("   • Cache preprocessed images when possible")
    
    print("\n2. Workflow Optimization:")
    print("   • Parallel processing for multiple CAPTCHAs")
    print("   • Asynchronous operations where possible")
    print("   • Intelligent retry strategies")
    print("   • Connection pooling and reuse")

# Display production guidelines
show_production_guidelines()

## Tutorial Summary

Comprehensive summary of the Strands + AgentCore + Bedrock CAPTCHA handling tutorial.

In [ ]:
def display_enhanced_tutorial_summary():
    """Display comprehensive tutorial summary with AI integration"""
    
    print("🎯 STRANDS + AGENTCORE + BEDROCK AI TUTORIAL COMPLETE")
    print("=" * 70)
    
    print("\n✅ WHAT YOU'VE LEARNED:")
    
    print("\n🧠 AI-Powered CAPTCHA Solving:")
    print("  • Bedrock Vision models for intelligent CAPTCHA analysis")
    print("  • Multi-modal AI understanding of text and image CAPTCHAs")
    print("  • Confidence scoring and solution validation")
    print("  • Adaptive retry strategies with enhanced models")
    
    print("\n🔒 Enterprise Security:")
    print("  • VM-level isolation for each CAPTCHA session")
    print("  • No data leakage between browser sessions")
    print("  • Enterprise-grade security compliance")
    print("  • Secure AI model access through Bedrock")
    
    print("\n📈 Serverless Scaling:")
    print("  • Automatic scaling from 1 to thousands of sessions")
    print("  • No infrastructure management required")
    print("  • Consumption-based pricing model")
    print("  • Zero idle resource costs")
    
    print("\n📊 Built-in Observability:")
    print("  • Session replay for debugging CAPTCHA failures")
    print("  • CloudTrail logging for compliance and audit")
    print("  • Real-time monitoring and metrics")
    print("  • AI model performance tracking")
    
    print("\n🤖 Strands Integration:")
    print("  • Native AgentCore Browser Tool support")
    print("  • Seamless tool and workflow integration")
    print("  • Production-ready error handling patterns")
    print("  • Enterprise deployment capabilities")
    
    print("\n💡 KEY ADVANTAGES vs Traditional Solutions:")
    print("  ✅ AI-powered analysis with state-of-the-art vision models")
    print("  ✅ No browser infrastructure to manage")
    print("  ✅ Enterprise-grade isolation and security")
    print("  ✅ Built-in observability and compliance features")
    print("  ✅ Automatic scaling without configuration")
    print("  ✅ Session replay for debugging")
    print("  ✅ Consumption-based cost model")
    print("  ✅ AWS-managed security and updates")
    print("  ✅ Confidence scoring and intelligent retry logic")
    
    print("\n🚀 NEXT STEPS:")
    print("\n1. Production Deployment:")
    print("   • Set up AWS credentials and AgentCore permissions")
    print("   • Configure Bedrock model access and quotas")
    print("   • Implement monitoring and alerting")
    print("   • Set up session pools for your workload")
    
    print("2. Advanced AI Features:")
    print("   • Fine-tune confidence thresholds for your use cases")
    print("   • Implement custom CAPTCHA detection algorithms")
    print("   • Add multi-region deployment for global scale")
    print("   • Integrate with custom vision models if needed")
    
    print("3. Compliance and Security:")
    print("   • Review CloudTrail logs for audit requirements")
    print("   • Implement data retention policies")
    print("   • Set up compliance monitoring dashboards")
    print("   • Regular security assessments and updates")
    
    print("\n🎉 Congratulations!")
    print("You're ready to build enterprise-grade AI-powered CAPTCHA automation with:")
    print("   🧠 State-of-the-art AI vision models")
    print("   🔒 VM-level isolation")
    print("   📈 Serverless auto-scaling")
    print("   📊 Built-in observability")
    print("   🤖 Strands framework integration")
    print("   ⚡ Intelligent confidence scoring")
    print("\n🚀 Start building your production AI CAPTCHA handling solution today!")

# Display the enhanced summary
display_enhanced_tutorial_summary()